In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# import sklearn
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.feature_extraction.text import TfidfVectorizer  # TF-IDF
from sklearn.preprocessing import scale
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import NMF
from sklearn.metrics.pairwise import cosine_similarity

import nltk
from nltk.corpus import stopwords 
from nltk.stem import LancasterStemmer, PorterStemmer
from nltk.stem import WordNetLemmatizer
import spacy #for faster tokenization and lemmatization

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import re
import string

#import project4_functions - rewrite?

%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
#also stored in sqlite database?

## Load and explore data

In [3]:
bbcdata = pd.read_json('/Users/xinrucheng/Documents/Metis_bootcamp/week_9/project5data/2017_140k/bbccouk-recipes.json', lines = True)
#ValueError: Trailing data
#JSON file has multiple lines, lines = True, or for loop:
#with open('your.json', 'rb') as f:
#    data = f.readlines()

In [4]:
bbcdata.shape
#has more recipes than image set! 10600, only 2225 have images
#photo url doesn't work, but recipe url does

(10599, 17)

In [5]:
bbcdata.head() 

,chef,chef_id,cooking_time_minutes,description,error,ingredients,instructions,instructions_detailed,photo_url,preparation_time_minutes,program,program_id,serves,time_scraped,title,total_time_minutes,url
0,James Tanner,james_tanner,10,,False,"[250g/8¾oz plain flour, plus extra for dusting...","[Preheat the oven to 200C/400F/Gas 6., For the...","[{'ingredient': 'plain flour', 'line': '250g/8...",None,30,Ready Steady Cook,b006vcgr,1,1499227761,Ten-minute pizza,40,http://bbc.co.uk/food/recipes/10minutepizza_87314
1,Mary Berry,mary_berry,0,This is my standby pasta supper as it is so de...,False,"[350g/12oz penne pasta, 2 x 80g/3oz packs Parm...",[Cook the pasta in a pan of boiling salted wat...,"[{'ingredient': 'pasta', 'line': '350g/12oz pe...",http://ichef.bbci.co.uk/food/ic/food_16x9_608/...,30,Mary Berry Cooks,p01s4q10,6,1499227763,15 minute pasta,30,http://bbc.co.uk/food/recipes/15_minute_pasta_...
2,Mari Williams,mari_williams,0,Simple 3D iced biscuits inspired by Bake Off. ...,False,"[200g/7oz unsalted soft butter, 200g/7oz caste...","[To make the basic dough, line a baking tray w...","[{'ingredient': 'butter', 'line': '200g/7oz un...",http://ichef.bbci.co.uk/food/ic/food_16x9_608/...,30,The Great British Bake Off,b013pqnm,0,1499227766,3D biscuits,30,http://bbc.co.uk/food/recipes/3d_biscuits_29555
3,Justine Pattison,justine_pattison,0,This easy turkey crown recipe is served with s...,False,[2.5kg/5lb 8oz turkey crown (fully thawed if f...,"[Preheat the oven to 220C/200C Fan/Gas 7., For...","[{'ingredient': 'turkey', 'line': '2.5kg/5lb 8...",http://ichef.bbci.co.uk/food/ic/food_16x9_608/...,30,None,None,6,1499227765,2-hour Christmas dinner,30,http://bbc.co.uk/food/recipes/2_hour_christmas...
4,Ainsley Harriott,ainsley_harriott,0,Rib-eye is one of the most flavoursome cuts of...,False,"[2.25kg/5lb rib-eye of beef, boned and rolled,...",[Place the rib-eye of beef into a large non-me...,"[{'ingredient': None, 'line': '2.25kg/5lb rib-...",None,120,Great British Food Revival,b016pbs9,8,1499227769,Mustard and thyme crusted rib-eye of beef,120,http://bbc.co.uk/food/recipes/_81487


In [6]:
bbcdata.info() #no nulls in text columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10599 entries, 0 to 10598
Data columns (total 17 columns):
chef                        10599 non-null object
chef_id                     10599 non-null object
cooking_time_minutes        10599 non-null int64
description                 10599 non-null object
error                       10599 non-null bool
ingredients                 10599 non-null object
instructions                10599 non-null object
instructions_detailed       10599 non-null object
photo_url                   2225 non-null object
preparation_time_minutes    10599 non-null int64
program                     9626 non-null object
program_id                  9626 non-null object
serves                      10599 non-null int64
time_scraped                10599 non-null int64
title                       10599 non-null object
total_time_minutes          10599 non-null int64
url                         10599 non-null object
dtypes: bool(1), int64(5), object(11)
memory usage:

In [7]:
#drop columns, only keep title, description, ingredients, instructions, instructions_detailed (has ingredients) and url for now

In [8]:
bbcdata.iloc[1]['instructions_detailed'] #looks like ingredients? saved as list of dict

[{'ingredient': 'pasta', 'line': '350g/12oz penne pasta'},
 {'ingredient': 'ham',
  'line': '2 x 80g/3oz packs Parma ham, snipped into small pieces'},
 {'ingredient': 'chestnut mushrooms',
  'line': '250g/9oz small brown chestnut mushrooms, halved or quartered'},
 {'ingredient': 'crème fraîche', 'line': '200g/7oz full-fat crème fraîche'},
 {'ingredient': 'Parmesan', 'line': '100g/3½oz Parmesan, grated'},
 {'ingredient': 'parsley', 'line': '2 tbsp chopped parsley'},
 {'ingredient': 'pepper', 'line': 'salt and pepper, to taste'},
 {'ingredient': 'salad', 'line': 'green salad'},
 {'ingredient': 'bread', 'line': 'crunchy bread'}]

In [9]:
bbcdata.iloc[1]['ingredients'] #easier to clean than the above, use this column

['350g/12oz penne pasta',
 '2 x 80g/3oz packs Parma ham, snipped into small pieces',
 '250g/9oz small brown chestnut mushrooms, halved or quartered',
 '200g/7oz full-fat crème fraîche',
 '100g/3½oz Parmesan, grated',
 '2 tbsp chopped parsley',
 'salt and pepper, to taste',
 'green salad',
 'crunchy bread']

In [10]:
bbcdata.iloc[1]['instructions'] 

['Cook the pasta in a pan of boiling salted water according to the packet instructions. Drain and set aside',
 'Heat a frying pan until hot. Add the pieces of Parma ham and fry until crisp, remove half of the ham onto a plate and set aside. Add the mushrooms to the pan and fry for two minutes. Add the crème fraîche and bring up to the boil. Add the pasta, Parmesan and parsley and toss together over the heat. Season well with salt and pepper.',
 'Serve with a green salad and crunchy bread.']

## Get topics 
### Vectorize recipe titles

In [11]:
titles = bbcdata['title']
type(titles)

pandas.core.series.Series

In [12]:
#remove digits
def regex_nodigits(s):
    '''use regex to clean string: 
    get rid of numbers, punctuations and capitalized letters''' 
    s = re.sub(r'[\d]','',str(s).lower())#convert to lower case, replace digits with empty str
    s = re.sub(r'[^a-z]',' ',str(s))#replace puctuations with space
    return s 

In [13]:
cleaned_t=titles.apply(regex_nodigits)
cleaned_t.head()

0                              ten minute pizza
1                                  minute pasta
2                                   d biscuits 
3                         hour christmas dinner
4    mustard and thyme crusted rib eye of beef 
Name: title, dtype: object

#### Count Vectorizer

In [14]:
#countVectorizer 
#Converts a collection of text documents to a matrix of token counts
#input is expected to be the sequence strings or bytes items are expected to be analyzed directly.

In [15]:
#start with standard nltk stopwords
stopwords_nltk = stopwords.words('english')

In [16]:
corpus = cleaned_t
ct_vectorizer = CountVectorizer(stop_words=stopwords_nltk)

titles_ct = ct_vectorizer.fit_transform(corpus)
print(ct_vectorizer.get_feature_names())

['aberdeen', 'abundant', 'acacia', 'ackee', 'adaptable', 'ade', 'adhraki', 'adobo', 'adraki', 'aduki', 'affogato', 'african', 'aged', 'aglio', 'agnello', 'ahead', 'aioli', 'ais', 'aji', 'ajillo', 'ajitsuke', 'ajo', 'ajwain', 'al', 'alaska', 'albanian', 'albert', 'albondigas', 'alcoholic', 'ale', 'alert', 'alfredo', 'alla', 'alle', 'alloro', 'allspice', 'almond', 'almondine', 'almonds', 'almondy', 'aloo', 'alpine', 'alsatian', 'alternative', 'amann', 'amaretti', 'amaretto', 'amarillo', 'amatriciana', 'amazing', 'ambassadeur', 'ambassador', 'amber', 'american', 'amoul', 'anatra', 'ancho', 'anchovade', 'anchovies', 'anchovy', 'ancienne', 'andalucian', 'angel', 'angela', 'anglais', 'anglaise', 'anglesey', 'anguilla', 'anise', 'aniseed', 'annika', 'anolini', 'antony', 'anya', 'aperitivo', 'apple', 'apples', 'apricot', 'apricots', 'arabian', 'aragon', 'arancine', 'arancini', 'arbequina', 'arbroath', 'arctic', 'arlettes', 'armagnac', 'arnaise', 'arnold', 'aromatic', 'arrabbiata', 'arrabiata',

#### TF-IDF
TF-IDF = (Term Frequency) * (Inverse Document Frequency)

(how often word occurs in this doc) * inverse of (how often this word occurs in all documents)

In [17]:
tf_vectorizer = TfidfVectorizer(stop_words=stopwords_nltk)
titles_tfidf = tf_vectorizer.fit_transform(corpus)

print(tf_vectorizer.get_feature_names()) #resulting tokens very similar to above

['aberdeen', 'abundant', 'acacia', 'ackee', 'adaptable', 'ade', 'adhraki', 'adobo', 'adraki', 'aduki', 'affogato', 'african', 'aged', 'aglio', 'agnello', 'ahead', 'aioli', 'ais', 'aji', 'ajillo', 'ajitsuke', 'ajo', 'ajwain', 'al', 'alaska', 'albanian', 'albert', 'albondigas', 'alcoholic', 'ale', 'alert', 'alfredo', 'alla', 'alle', 'alloro', 'allspice', 'almond', 'almondine', 'almonds', 'almondy', 'aloo', 'alpine', 'alsatian', 'alternative', 'amann', 'amaretti', 'amaretto', 'amarillo', 'amatriciana', 'amazing', 'ambassadeur', 'ambassador', 'amber', 'american', 'amoul', 'anatra', 'ancho', 'anchovade', 'anchovies', 'anchovy', 'ancienne', 'andalucian', 'angel', 'angela', 'anglais', 'anglaise', 'anglesey', 'anguilla', 'anise', 'aniseed', 'annika', 'anolini', 'antony', 'anya', 'aperitivo', 'apple', 'apples', 'apricot', 'apricots', 'arabian', 'aragon', 'arancine', 'arancini', 'arbequina', 'arbroath', 'arctic', 'arlettes', 'armagnac', 'arnaise', 'arnold', 'aromatic', 'arrabbiata', 'arrabiata',

#### LSA on CtVec titles


In [18]:
lsa_10 = TruncatedSVD(10)  #Dimensionality reduction using truncated SVD (aka LSA).
#10 topics
t_lsa_10 = lsa_10.fit_transform(titles_ct)
lsa_10.explained_variance_ratio_.sum()
#explained variance ratio too low? .sum() only 12%
#only 46% for 100 topics! need to clean further to get higher variance ration with a reasonable number of topics?

0.11877882891003717

In [19]:
#from proj4 and topic modelling/LSA/NMF lecture
def display_topics(model, feature_names, no_top_words, topic_names=None):
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix)
        else:
            print("\nTopic: '",topic_names[ix],"'")
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [20]:
display_topics(lsa_10, ct_vectorizer.get_feature_names(), 10) 


Topic  0
sauce, salad, chicken, fried, roasted, tomato, potato, cheese, chilli, roast

Topic  1
sauce, chocolate, cream, wine, pork, fried, pudding, ice, cake, roast

Topic  2
chicken, roast, fried, rice, curry, stuffed, stir, pie, garlic, potatoes

Topic  3
chocolate, chicken, cream, salad, cake, ice, pudding, orange, white, mousse

Topic  4
cream, chocolate, roasted, potato, lamb, cheese, cake, sweet, pudding, soup

Topic  5
fried, pan, chocolate, deep, egg, cream, stir, rice, chilli, salmon

Topic  6
cheese, potato, goats, tomato, soup, sweet, cream, blue, onion, baked

Topic  7
cheese, tomato, lamb, roasted, chocolate, goats, potatoes, stuffed, garlic, tomatoes

Topic  8
chocolate, potato, soup, sweet, chilli, red, beef, lamb, tomato, onion

Topic  9
pork, cheese, roasted, apple, chocolate, goats, belly, fried, salad, cabbage


In [21]:
#topics all over the place, and most have chocolate!

#try TF-IDF, NMF, *LDA and spacy?

#### LSA on titles vectorized by TF-IDF

In [22]:
lsa_10 = TruncatedSVD(10)  #10 topics
t_tf_lsa_10 = lsa_10.fit_transform(titles_tfidf)
lsa_10.explained_variance_ratio_

array([0.0046158 , 0.00769671, 0.00703018, 0.00662758, 0.00643933,
       0.00570901, 0.00548669, 0.00539797, 0.00516311, 0.00497743])

In [23]:
display_topics(lsa_10, tf_vectorizer.get_feature_names(), 10) 


Topic  0
salad, chicken, sauce, fried, tomato, potato, roasted, chilli, cheese, soup

Topic  1
chocolate, sauce, cake, cream, pudding, orange, ice, mousse, white, raspberry

Topic  2
chicken, roast, lemon, pie, curry, sauce, rice, fried, chocolate, stir

Topic  3
soup, potato, sweet, sauce, pork, roast, leek, pea, roasted, red

Topic  4
soup, chicken, chocolate, potato, cake, pie, salad, cheese, cream, leek

Topic  5
cheese, goats, roast, sauce, stuffed, lamb, roasted, pie, souffl, potatoes

Topic  6
tomato, sauce, soup, chicken, chocolate, salad, pasta, spinach, basil, white

Topic  7
cheese, fried, goats, chilli, souffl, rice, pan, stir, egg, blue

Topic  8
pudding, tomato, apple, bread, butter, pie, soup, toffee, cream, rice

Topic  9
lamb, chilli, roasted, garlic, chocolate, potatoes, roast, soup, cake, squash


In [24]:
#Topic 1 is dessert, still messy though - bi-grams? or use instructions instead - lemmatize

In [25]:
#5 topics?
lsa_5 = TruncatedSVD(5)  #5 topics
t_tf_lsa_5 = lsa_5.fit_transform(titles_tfidf)
lsa_5.explained_variance_ratio_
#lsa_5.explained_variance_ratio_.sum() #only 3% of variance!

array([0.00461168, 0.00769694, 0.00703141, 0.00663835, 0.00642146])

In [26]:
display_topics(lsa_5, tf_vectorizer.get_feature_names(), 10) #fewer topics, cutoff earlier, same as top 5 topics in 10 topic case


Topic  0
salad, chicken, sauce, fried, potato, tomato, roasted, chilli, cheese, soup

Topic  1
chocolate, sauce, cake, cream, pudding, orange, ice, mousse, white, raspberry

Topic  2
chicken, curry, lemon, roast, pie, chocolate, rice, stir, fried, sauce

Topic  3
soup, sauce, potato, sweet, fried, pork, roast, red, leek, roasted

Topic  4
soup, chicken, potato, chocolate, cheese, cake, cream, pudding, leek, sweet


#### CtVec+ NMF

In [27]:
nmf_10 = NMF(10) 
t_nmf_10 = nmf_10.fit_transform(titles_ct) 

In [28]:
display_topics(nmf_10, ct_vectorizer.get_feature_names(), 20)


Topic  0
sauce, tomato, wine, red, apple, fillet, steak, butter, roast, pork, mushroom, poached, duck, wild, vierge, pasta, mustard, ed, toffee, potatoes

Topic  1
salad, dressing, warm, tomato, salmon, bean, herb, smoked, spinach, green, chilli, fennel, thai, grilled, noodle, rocket, cucumber, beef, style, watercress

Topic  2
chicken, roast, curry, rice, lemon, stuffed, pie, chilli, breast, thai, liver, chorizo, spiced, spicy, mushroom, garlic, honey, style, stir, potatoes

Topic  3
chocolate, cake, orange, white, mousse, pudding, raspberry, hot, cheesecake, caramel, souffl, tart, chilli, brownies, fondant, coffee, baked, bread, dark, sponge

Topic  4
roasted, pork, tomatoes, potatoes, red, pepper, apple, chilli, stuffed, lemon, cabbage, garlic, honey, belly, vegetables, sea, slow, onion, squash, duck

Topic  5
fried, pan, egg, deep, stir, rice, chilli, salmon, sea, bass, mushrooms, beef, tomato, spicy, onion, salsa, broccoli, wild, noodles, courgette

Topic  6
cheese, goats, tomato

For this dataset, NMF topics seem to make more sense (somewhat divided) than LSA ones (chocolate in every topic). 
Can still improve before vectorization and dim reduction though

CtVec + NMF: 
* Topic 0 steak, pasta, roast
* Topic 1 salad, vegetables
* Topic 2 curry, rice
* Topic 3 desserts, chocolate, coffee (darker colour?)
* Topic 4 roast
* Topic 5 pan-fried, eggs and fish
* Topic 6 cheese, baked
* Topic 7 potatoe, soup
* Topic 8 desserts, ice cream, tart, cake (lighter colours)
* Topic 9 lamb roast

#### TFIDF+ NMF

In [29]:
nmf_10 = NMF(10) 
t_tf_nmf_10 = nmf_10.fit_transform(titles_tfidf) 

In [30]:
display_topics(nmf_10, tf_vectorizer.get_feature_names(), 20)


Topic  0
salad, dressing, warm, salmon, bean, smoked, herb, spinach, tomato, noodle, bacon, cucumber, fennel, mango, green, thai, rocket, orange, mackerel, style

Topic  1
chocolate, cake, cream, orange, mousse, white, ice, raspberry, tart, cheesecake, souffl, hot, lemon, caramel, strawberry, vanilla, brownies, fruit, coffee, hazelnut

Topic  2
chicken, curry, lemon, pie, thai, roast, liver, rice, chorizo, mushroom, stuffed, casserole, style, breast, skewers, korma, coconut, jerk, spicy, noodles

Topic  3
sauce, tomato, pasta, wine, mushroom, basil, meatballs, red, poached, steak, pancakes, vierge, cherry, apple, fillet, creamy, wild, fish, cream, bass

Topic  4
soup, pea, spinach, leek, noodle, cream, mushroom, vegetable, broccoli, squash, butternut, mint, cro, creamy, tons, stilton, poached, lentil, parsnip, sweetcorn

Topic  5
fried, pan, chilli, stir, egg, salmon, rice, deep, beef, fry, smoked, broccoli, sea, noodles, prawns, poached, bass, mushrooms, spicy, squid

Topic  6
potato

For TFIDF and NMF, topics 1 and 8 are desserts, Topic 4 is a bit strange (salmon and meringue)

* Topic 0 salad
* Topic 1 dessert, chocolate
* Topic 2 curry
* Topics 3 soup, vegetarian
* Topic 4 dessert? (except for sole and salmon)
* Topic 5 pan-fried, eggs and fish, noodles
* Topic 6 roast - pork, lamb, steak, duck
* Topic 7 cheese, roasted, baked, pizza
* Topic 8 dessert, rice pudding, toffee, sponge cake (non-chocolate)
* Topic 9 potatoes, meat pies and stews


In [31]:
#try LDA later?
#better cleaning and lemmatization?
#or can use these for recommendations already?

In [32]:
#how to group topics for filtering? cluster?

### Vectorize recipe instructions

Now it seems it would make more sense to lemmatize 'instructions' column and use those for topics (include both ingredients and cooking process, which can imply on the course of meal (boil, pan-fried less likely to be desserts than bake);

instructions give the most information, contains ingredients, and can imply the course of meal

In [33]:
#use tfidf + NMF (most clearly divided)
instructions = bbcdata['instructions']
instructions.head()

0    [Preheat the oven to 200C/400F/Gas 6., For the...
1    [Cook the pasta in a pan of boiling salted wat...
2    [To make the basic dough, line a baking tray w...
3    [Preheat the oven to 220C/200C Fan/Gas 7., For...
4    [Place the rib-eye of beef into a large non-me...
Name: instructions, dtype: object

#### Clean text with regex

In [34]:
#clean text with regex, ftn defined above
# def regex_nodigits(s):
#     '''use regex to clean string: 
#     get rid of numbers, punctuations and capitalized letters''' 
#     s = re.sub(r'[\d]','',str(s).lower())#convert to lower case, replace digits with empty str
#     s = re.sub(r'[\W]','',str(s))#replace puctuations with empty str
#     return s 

In [35]:
cld_ins=instructions.apply(regex_nodigits)
cld_ins.head()

0      preheat the oven to c f gas      for the piz...
1      cook the pasta in a pan of boiling salted wa...
2      to make the basic dough  line a baking tray ...
3      preheat the oven to c c fan gas      for the...
4      place the rib eye of beef into a large non m...
Name: instructions, dtype: object

In [36]:
cld_ins[0]

'  preheat the oven to c f gas      for the pizza base  place the flour  oil  water and salt into a food processor and blend together until a dough is formed  tip out onto a floured work surface and knead  shape into a round base about cm in wide     place into a frying pan over a high heat and brown the base  then using a mini blowtorch  crisp the top of the pizza   alternatively you can do this under the grill      for the topping  spread tomato pur e over the top of the base     fry the mushrooms in a dry frying pan then scatter over the tomato pur e  arrange the prosciutto and cheese on top     crack an egg into the middle  then place into the oven for five minutes to finish cooking     serve on a large plate  and slice into wedges to serve   '

#### TF-IDF + NMF

In [37]:
corpus = cld_ins
tf_vectorizer = TfidfVectorizer(stop_words=stopwords_nltk)
instr_tfidf = tf_vectorizer.fit_transform(corpus)

print(tf_vectorizer.get_feature_names()) 

#too many repetitions, need stemming and lemmatization!

['aa', 'abaking', 'abandon', 'abd', 'ability', 'able', 'aboutcm', 'absolute', 'absolutely', 'absorb', 'absorbed', 'absorbency', 'absorbent', 'absorbing', 'absorbs', 'absorption', 'abundant', 'aburaage', 'accelerate', 'acceptable', 'accessible', 'accidentally', 'accommodate', 'accompainment', 'accompanied', 'accompaniment', 'accompaniments', 'accompany', 'accord', 'according', 'accordingly', 'accorning', 'account', 'accross', 'accumulate', 'accumulated', 'accumulates', 'accurate', 'accurately', 'acetate', 'achieve', 'achieved', 'achieves', 'acid', 'acidic', 'acidity', 'acidulated', 'ackee', 'acm', 'acorn', 'acorns', 'acquire', 'acrid', 'across', 'act', 'acting', 'action', 'activate', 'activated', 'active', 'activity', 'actual', 'actually', 'ad', 'add', 'adddrop', 'added', 'adding', 'addition', 'additional', 'additions', 'adds', 'addtablespoons', 'addtbsp', 'addthe', 'adhere', 'adheres', 'adhering', 'adhesive', 'aditional', 'adjacent', 'adjoining', 'adjust', 'adjusting', 'adjustments', '

In [38]:
nmf_10 = NMF(10) 
instr_tf_nmf_10 = nmf_10.fit_transform(instr_tfidf) 

In [39]:
display_topics(nmf_10, tf_vectorizer.get_feature_names(), 20) 
#without stemming/lemmatization, topics dominated by baking instructions!


Topic  0
add, heat, minutes, cook, pan, stir, fry, simmer, rice, sauce, stock, boil, bring, garlic, onion, water, medium, oil, frying, saucepan

Topic  1
sugar, whisk, cream, mixture, chocolate, egg, place, bowl, milk, oven, pour, whites, vanilla, custard, heat, peaks, butter, fold, meringue, yolks

Topic  2
pastry, pie, tart, filling, oven, roll, brush, case, egg, edges, baking, cm, bake, beaten, surface, minutes, top, cut, tin, gas

Topic  3
place, pepper, freshly, black, ground, olive, season, salt, salad, oil, dressing, drizzle, oven, plate, pan, onto, cheese, bowl, serve, salmon

Topic  4
chicken, breast, breasts, cooked, marinade, pieces, juices, minutes, oven, thighs, skin, clear, sauce, rice, run, place, roast, side, add, roasting

Topic  5
dough, flour, surface, roll, knead, pasta, baking, floured, work, lightly, tray, yeast, bowl, mix, water, minutes, cm, together, place, oven

Topic  6
oil, deep, hot, kitchen, batter, unattended, dangerous, caution, drain, paper, slotted, g

* Topic 0 rice stir fry
* Topic 1 dessert - cream, chocolate, meringue
* Topic 2 dessert - pastry/pie/tart
* Topic 3 salmon, cheese and pepper? need more cleaning
* Topic 4 chicken
* Topic 5 dough, yeast, pasta
* Topic 6 battered and fried (fish sticks?)
* Topic 7 pork roast
* Topic 8 lamb roast
* Topic 9 cake

In [40]:
#more cleaning; 
#also - only use the recipes with images? purpose is to filter! (might get better topics?)

### Focus on smaller set of recipes
Only the 2225 recipes with images, to use as filter for image data

In [41]:
bbcdata.head()

,chef,chef_id,cooking_time_minutes,description,error,ingredients,instructions,instructions_detailed,photo_url,preparation_time_minutes,program,program_id,serves,time_scraped,title,total_time_minutes,url
0,James Tanner,james_tanner,10,,False,"[250g/8¾oz plain flour, plus extra for dusting...","[Preheat the oven to 200C/400F/Gas 6., For the...","[{'ingredient': 'plain flour', 'line': '250g/8...",None,30,Ready Steady Cook,b006vcgr,1,1499227761,Ten-minute pizza,40,http://bbc.co.uk/food/recipes/10minutepizza_87314
1,Mary Berry,mary_berry,0,This is my standby pasta supper as it is so de...,False,"[350g/12oz penne pasta, 2 x 80g/3oz packs Parm...",[Cook the pasta in a pan of boiling salted wat...,"[{'ingredient': 'pasta', 'line': '350g/12oz pe...",http://ichef.bbci.co.uk/food/ic/food_16x9_608/...,30,Mary Berry Cooks,p01s4q10,6,1499227763,15 minute pasta,30,http://bbc.co.uk/food/recipes/15_minute_pasta_...
2,Mari Williams,mari_williams,0,Simple 3D iced biscuits inspired by Bake Off. ...,False,"[200g/7oz unsalted soft butter, 200g/7oz caste...","[To make the basic dough, line a baking tray w...","[{'ingredient': 'butter', 'line': '200g/7oz un...",http://ichef.bbci.co.uk/food/ic/food_16x9_608/...,30,The Great British Bake Off,b013pqnm,0,1499227766,3D biscuits,30,http://bbc.co.uk/food/recipes/3d_biscuits_29555
3,Justine Pattison,justine_pattison,0,This easy turkey crown recipe is served with s...,False,[2.5kg/5lb 8oz turkey crown (fully thawed if f...,"[Preheat the oven to 220C/200C Fan/Gas 7., For...","[{'ingredient': 'turkey', 'line': '2.5kg/5lb 8...",http://ichef.bbci.co.uk/food/ic/food_16x9_608/...,30,None,None,6,1499227765,2-hour Christmas dinner,30,http://bbc.co.uk/food/recipes/2_hour_christmas...
4,Ainsley Harriott,ainsley_harriott,0,Rib-eye is one of the most flavoursome cuts of...,False,"[2.25kg/5lb rib-eye of beef, boned and rolled,...",[Place the rib-eye of beef into a large non-me...,"[{'ingredient': None, 'line': '2.25kg/5lb rib-...",None,120,Great British Food Revival,b016pbs9,8,1499227769,Mustard and thyme crusted rib-eye of beef,120,http://bbc.co.uk/food/recipes/_81487


#### drop rows where selected column is NaN

In [42]:
#drop all the rows where photo_url = None (cell is NoneType, not string 'None') - can just use dropna
#withphotos = bbcdata[bbcdata['photo_url'] == 'None'] 

In [43]:
#df.dropna(subset=[1]) #drop rows only if column 1 has nan**
withphotos = bbcdata.dropna(subset=['photo_url'])
withphotos.shape

(2225, 17)

#### Clean and vectorize instructions for 2k recipes with photos

In [44]:
cld_withphotos=withphotos['instructions'].apply(regex_nodigits)
cld_withphotos.head()

1       cook the pasta in a pan of boiling salted wa...
2       to make the basic dough  line a baking tray ...
3       preheat the oven to c c fan gas      for the...
7       preheat the oven to c f gas      take a very...
18      preheat the oven to c c fan gas      heat g ...
Name: instructions, dtype: object

In [45]:
nltkstopwords = stopwords.words('english')
len(nltkstopwords)

179

In [46]:
nltkstopwords.extend(['minutes','add','remove'])
len(nltkstopwords)

182

In [47]:
corpus = cld_withphotos
tf_vectorizer = TfidfVectorizer(stop_words=nltkstopwords) #using extended stopwords
tokens_tfidf = tf_vectorizer.fit_transform(corpus)

print(tf_vectorizer.get_feature_names()) #add stemmer too?

['abd', 'able', 'absolute', 'absolutely', 'absorb', 'absorbed', 'absorbency', 'absorbent', 'absorbs', 'accommodate', 'accompanied', 'accompaniment', 'according', 'accordingly', 'accumulated', 'accurately', 'acetate', 'achieve', 'achieved', 'achieves', 'acid', 'acidity', 'ackee', 'acquire', 'across', 'act', 'action', 'active', 'actual', 'actually', 'added', 'adding', 'addition', 'additional', 'additions', 'adds', 'adhere', 'adheres', 'adjust', 'adjusting', 'aduki', 'adult', 'adults', 'advance', 'advocaat', 'aerated', 'aesthetic', 'affect', 'afraid', 'agar', 'agave', 'agents', 'agitate', 'ahead', 'aim', 'aiming', 'aioli', 'air', 'airily', 'airing', 'airtight', 'airy', 'ajar', 'al', 'alarmed', 'alaska', 'albondigas', 'alcohol', 'alcoholic', 'ale', 'alight', 'alla', 'allow', 'allowed', 'allowing', 'allows', 'allspice', 'almond', 'almonds', 'almost', 'alone', 'along', 'alongside', 'already', 'also', 'alter', 'alternate', 'alternately', 'alternating', 'alternative', 'alternatively', 'althoug

In [48]:
#more cleaning! stemming?
#TFIDF
nmf_10 = NMF(10) 
withphotos_tf_nmf_10 = nmf_10.fit_transform(tokens_tfidf) 

#### **NMF topics**

In [49]:
display_topics(nmf_10, tf_vectorizer.get_feature_names(), 20) 


Topic  0
cook, heat, simmer, stirring, bring, boil, stock, stir, water, saucepan, rice, onion, sauce, pan, pasta, reduce, tender, lid, garlic, season

Topic  1
chocolate, sugar, whisk, mixture, cream, bowl, egg, set, cool, baking, oven, butter, pour, melted, meringue, place, heat, milk, vanilla, water

Topic  2
pastry, pie, filling, roll, tart, case, egg, brush, oven, edges, tin, cm, beaten, baking, fridge, bake, cut, mixture, line, surface

Topic  3
dough, flour, surface, baking, knead, roll, floured, work, bowl, place, lightly, yeast, leave, tray, onto, oven, mix, together, size, shape

Topic  4
chicken, marinade, breasts, cooked, pieces, thighs, breast, juices, marinate, place, ingredients, clear, cover, stock, skin, sauce, pan, barbecue, rice, run

Topic  5
pepper, salad, olive, place, dressing, season, salt, bowl, lemon, oil, mix, black, drizzle, freshly, juice, fish, ground, well, ingredients, taste

Topic  6
cake, tin, icing, beat, cakes, cool, sugar, cm, skewer, oven, tins, to

Topics look good!

tfidf+nmf with extended stopwords (minutes, add, remove)

* Topic 0 roast - potatoes, lamb, beef, pork
* Topic 1 dessert - chocolate, meringue
* Topic 2 dessert - pastry, pie, tart
* Topic 3 bread (dough, yeast, knead, roll, shape - no sugar)
* Topic 4 chicken
* Topic 5 salad
* Topic 6 cake
* Topic 7 Asian - curry, rice, stir-fry
* Topic 8 pasta
* Topic 9 pan-fry (european?)

    (Previously, tfidf+ nmf, 10 topics without extended stopwords )
    * Topic 0 rice stir fry
    * Topic 1 dessert - chocolate, meringue
    * Topic 2 dessert - pastry, pie, tart
    * Topic 3 dough, yeast (bread?)
    * Topic 4 chicken
    * Topic 5 salad
    * Topic 6 cake
    * Topic 7 Asian, noodles (wok, fry, soy, seasame) 
    * Topic 8 pasta, spaghetti
    * Topic 9 potatoes, lamb roast, beef, pork

In [50]:
#restarted kernel, order of topics changed? one topic content changed also, the rest were the sam

#next? more cleaning? - topics look good, except for stir and stirring in topic 0, tin and tins in Topic 6, not much need for stemmer
    #can still add stemmer to improve it
    #also, can add minute/minutes/hours to stopwords
    
#cluster on topics? in order to assign new image to cluster - can we expect the clusters to match the topics?

#Clustering - if use Kmeans, need to convert and approx cosine similarity
    #then predict which cluster the image belongs to; 
    # also need to predict/assign cluster to each image in dataset (could be slow)
    # Kmeans assigns one doc to one cluster only, "hard"
    # "Fuzzy Kmeans" gives probability (documentation minimal, run on cmd line?)
#should be able to do this with topics also?
    # topic modelling, each document has topic weights, can assign to max-weighted topic; LDA?
    

#### Get doc-topic matrix 
to use NMF topics

In [51]:
withphotos.title

1                                          15 minute pasta
2                                             3D biscuits 
3                                  2-hour Christmas dinner
7        A classic sponge cake (with passion fruit fill...
18               Albanian baked lamb with rice (Tavë kosi)
                               ...                        
10574                                   Yorkshire puddings
10576                           Yorkshire ‘tapas’ puddings
10582                                             Yule log
10585                             Za’atar cod with relish 
10589                                Zesty tofu cheesecake
Name: title, Length: 2225, dtype: object

##### *Set index to original index - *Not recipe ID!

In [52]:
#index = withphotos.title  #instead of changing index from ID to title, better to use ID, or keep both?
index = withphotos.index
columns = ['topic ' + str(num) for num in range(10)]

In [53]:
#withphotos_tf_nmf_10 = nmf_10.fit_transform(tokens_tfidf) 
doc_topic = pd.DataFrame(withphotos_tf_nmf_10.round(5),index,columns)
doc_topic

,topic 0,topic 1,topic 2,topic 3,topic 4,topic 5,topic 6,topic 7,topic 8,topic 9
1,0.03491,0.00000,0.00000,0.00000,0.00000,0.05341,0.00000,0.00000,0.00000,0.15217
2,0.00000,0.03898,0.00568,0.06684,0.00000,0.00000,0.04154,0.00000,0.02900,0.00000
3,0.05478,0.00824,0.00000,0.02322,0.00574,0.03262,0.00000,0.00000,0.17753,0.01710
7,0.00000,0.06505,0.00689,0.01557,0.00073,0.03536,0.09565,0.00641,0.00000,0.00000
18,0.06629,0.00546,0.00220,0.00000,0.00000,0.00519,0.00000,0.00000,0.16475,0.06193
...,...,...,...,...,...,...,...,...,...,...
10574,0.00000,0.05277,0.00224,0.02373,0.00000,0.00771,0.00681,0.00064,0.08445,0.01913
10576,0.03424,0.05877,0.01129,0.01175,0.05156,0.01469,0.01892,0.01663,0.05066,0.04952
10582,0.00000,0.05960,0.01715,0.02681,0.00000,0.00000,0.15664,0.00000,0.00000,0.00000
10585,0.00000,0.00000,0.00000,0.00647,0.01219,0.08390,0.00000,0.01125,0.00000,0.06765


#### *Assign document to topic with highest weight
*alternative is clustering in topic space, documents are points

In [57]:
#numpy.argmax -- Returns the indices of the maximum values along an axis. Default index is into the flattened array (axis=None, works on entire array)
#returns first occurrence if max occurs multiple times
#*may need to reshape?

#test on small df first
test = doc_topic.head()
test

,topic 0,topic 1,topic 2,topic 3,topic 4,topic 5,topic 6,topic 7,topic 8,topic 9
1,0.03491,0.00000,0.00000,0.00000,0.00000,0.05341,0.00000,0.00000,0.00000,0.15217
2,0.00000,0.03898,0.00568,0.06684,0.00000,0.00000,0.04154,0.00000,0.02900,0.00000
3,0.05478,0.00824,0.00000,0.02322,0.00574,0.03262,0.00000,0.00000,0.17753,0.01710
7,0.00000,0.06505,0.00689,0.01557,0.00073,0.03536,0.09565,0.00641,0.00000,0.00000
18,0.06629,0.00546,0.00220,0.00000,0.00000,0.00519,0.00000,0.00000,0.16475,0.06193


In [58]:
np.argmax(np.array(test), axis = 1) #works! #.reshape(5,10) not necessary in this case
#topics changed after changing index from title to ID?

array([9, 3, 8, 6, 8])

In [59]:
test.shape

(5, 10)

In [57]:
for row in test.transpose():
    print(row) #index is ID (prev index =title

1
2
3
7
18


In [60]:
test.transpose().iloc[:,0] #prints first row**
#test.transpose().iloc[0] prints the 0th column! 

topic 0    0.03491
topic 1    0.00000
topic 2    0.00000
topic 3    0.00000
topic 4    0.00000
topic 5    0.05341
topic 6    0.00000
topic 7    0.00000
topic 8    0.00000
topic 9    0.15217
Name: 1, dtype: float64

In [61]:
def assign_topic(df):
    '''For each row in df, finds the topic with highest weight, 
    assign that topic label to the row.
    Returns a dataframe''' #df easiest to access, will group recipes and images by labels later
    label_list = []
    for i in range(len(df)): #easier way to do this?
        topic_label = np.argmax(np.array(df.transpose().iloc[:,i]), axis = 0)
        label_list.append(topic_label)
    return pd.DataFrame(label_list, index = index, columns = ['topics'])#columns needs to be a list
    #better if make df outside of function?
    #np.argmax(np.array(test), axis = 1)
#edit ftn before pickling*

#index = withphotos.index

In [62]:
#assign_topic(test)
labelled = assign_topic(doc_topic)
labelled  #is 1 coln df enough? use other data structure? save as sparse matrix instead?**

,topics
1,9
2,3
3,8
7,6
18,8
...,...
10574,8
10576,1
10582,6
10585,5


In [69]:
labelled['idx'] = labelled.index  #*not necessary? this id is not the unique recipe ID*!
labelled

,topics,idx
1,9,1
2,3,2
3,8,3
7,6,7
18,8,18
...,...,...
10574,8,10574
10576,1,10576
10582,6,10582
10585,5,10585


In [62]:
#if use this data structure, need to loop through 10 times to save each topic label as a list? to make list of lists?
#instead, just look up topic label from df? (could be slow on app? but dataset not big)

#pseudocode: (more complicated than I expected?)

    #give each topic label a summary (2.3.3, 2-word max?), provide those to the user on app (drop down list? or type in box with drop-down)
    #write ftn that takes that word as input, and returns a list of all recipe titles of that label
    
    #another ftn: *combine labelled sublist with image data - match each recipe with its image 
        #(*what's the fastest way? image title has title with underscore and recipe ID - should keep ID from original data (not a column)?)
        #returns a list of images (titles?), use those to find features
        

##### **Label names of 10 topics:**

```
* Topic 0 roast - potatoes, lamb, beef, pork
* Topic 1 dessert - chocolate, meringue
* Topic 2 dessert - pastry, pie, tart
* Topic 3 bread 
* Topic 4 chicken
* Topic 5 salad
* Topic 6 cake
* Topic 7 Asian - curry, rice, stir-fry
* Topic 8 pasta
* Topic 9 pan fry
```
(combine the 2 dessert topics?)

In [63]:
idx_coln = withphotos.index
type(idx_coln) 

pandas.core.indexes.numeric.Int64Index

#### Find documents of the selected topic

In [70]:
def get_topic_num(name):
    '''For a given label name (out of 10 choices), returns the topic number'''
    label_names_dict = {'roast':0,
                   'dessert (chocolate)':1, 
                   'dessert (pastry, pie)':2, 
                   'bread':3,
                   'chicken':4,
                   'salad':5,
                   'cake':6,
                   'Asian':7,
                   'pasta':8,
                   'pan fry':9} #*change to floats for combined?
    # dictionary --keys=names, values=topic number; match label names with topic number in doc_topic and labelled df
    if name in label_names_dict: #check user input is key in dict
        topic_num = label_names_dict[name]#get matching values(topic number) 
        return topic_num #=6 for cake, works

def get_recipes_by_topic(num, data):
    '''Takes in the topic number (0 to 9),
    returns a list of IDs of all recipe titles assigned to that topic
    (those that have the label as the topic with maximum weight from NMF topic modelling)'''
    docs_same_label = []
    #result = [f(x) for x in df['col']] better and faster than iter ftns
    #result = [f(x, y) for x, y in zip(df['col1'], df['col2'])]
    for topicn, idn in zip(data['topics'], data['idx']):
        if topicn == num:
            docs_same_label.append(idn)
    return docs_same_label
#better to cluster than assigning?

In [65]:
get_topic_num('roast')

0

In [66]:
#check classic sponge cake recipe (topic 6, cake)
labelled.iloc[3] #name is index, can't use loc on it as a regular column?
#iloc[0], expect to get 8 for pasta?

topics    6
Name: 7, dtype: int64

In [71]:
cakes = get_recipes_by_topic(float(6), labelled)
#a list of all recipe ids that are cakes
len(cakes) #restart kernel, length increased?

150

In [73]:
#^^^debug
cakes[40]

2097

#### *Match docs with selected topic to their images

##### Clean image titles to extract IDs

In [68]:
#!problem: ID in image title is not the same as index in recipe? recipe id max at 10589 (10589 text recipes)
#but images have a much larger ID (dep on how many are scraped from website?)
#how to match recipes to image?

#in Cosine_Similarity code, using index in bbc_list (order of images in directory) 
    #In first nb-Feature_Extractor code, also following that order - but not matching because recipes with no images don't exist in features.csv
    #image index in FtExtr 0-2225, but how to get recipe withphotos text index (1,2,3,7,18) to each image? 


* clean images and use 4-5 digit number as recipe ID
* add column for URL to text df
* add extract the same recipe ID number as ID column to doc_topic and labelled dfs
* *add recipe ID column to FeatureExtractor code, extract features again (2k, <1hr locally?)*
* join id-topic text df with id-features image df
* can then filter recipe and images by topic!

*allrecipes set has ID as image title, can probably use same code to match those features with recipe text - is it worth it?*

In [74]:
bbcimg_list = !ls '/Users/xinrucheng/Documents/Metis_bootcamp/week_9/project5data/2017_140k/recipe_photos/bbc_photos/pages-photos/'
len(bbcimg_list) #doesn't work if use !ls img_path variable

2225

In [75]:
#example image title: _chicken_chasseur_with_19163_16x9.jpg
def clean_img_title(imglist):
    '''use regex to clean image titles to get unique recipe ID,
    imglist is a list of image title strings
    Returns a dataframe where the column ID is integer IDs, 
    and the column title is the original image title''' 
    import re
    ID_list = []
    title_list = []
    for filename in imglist:
        title_list.append(filename)
        #x = re.split("_", filename)
        ids = re.findall(r"_(\d+)_", str(filename))
        #print(ids)
        ID_list.append(ids)
        
    imgID_df = pd.DataFrame(ID_list, columns=['ID', 'title']) #1 col vs 2 col error* made extra column of None
    imgID_df['title'] = title_list

    return imgID_df

In [76]:
#testpath = '/Users/xinrucheng/Documents/Metis_bootcamp/week_9/metis_passion_project/data/raw/testimages/bbctest'

test_list = !ls '/Users/xinrucheng/Documents/Metis_bootcamp/week_9/metis_passion_project/data/raw/testimages/bbctest'
len(test_list)


12

In [77]:
test_list[0]

'baileysandchocolatec_72293_16x9.jpg'

In [73]:
#clean_img_title(test_list)

In [74]:
#regex notes
#re.findall returns a list of matches (Return an empty list if no match was found), better than re.search (returns a match object)
#regex - () for group it captures and returns! can still keep str structure outside of group to help it search
#https://regex101.com/#python  (regex expr above test str box, colour-coded)


In [78]:
bbcimg_list[0]

'15_minute_pasta_33407_16x9.jpg'

In [79]:
img_ids = clean_img_title(bbcimg_list) #worked for smaller test folder?, not sure where extra column comes from - bbc_list and test_list not the same dimensions
img_ids #column 0 is id, drop col 1 by index but not by name?

,ID,title
0,33407,15_minute_pasta_33407_16x9.jpg
1,79341,2_hour_christmas_dinner_79341_16x9.jpg
2,29555,3d_biscuits_29555_16x9.jpg
3,19163,_chicken_chasseur_with_19163_16x9.jpg
4,49934,_schichttorte_49934_16x9.jpg
...,...,...
2220,93848,yorkshirepudding_93848_16x9.jpg
2221,86010,yorkshirepuddings_86010_16x9.jpg
2222,15656,yule_log_15656_16x9.jpg
2223,98478,zaatar_cod_with_relish_98478_16x9.jpg


##### Get ID from URL column in text df
extract the same recipe ID number as ID column in 'doc_topic' df and 'labelled' df (2.3.4.1)

In [80]:
withphotos.head() #df with recipes that don't have photos dropped

,chef,chef_id,cooking_time_minutes,description,error,ingredients,instructions,instructions_detailed,photo_url,preparation_time_minutes,program,program_id,serves,time_scraped,title,total_time_minutes,url
1,Mary Berry,mary_berry,0,This is my standby pasta supper as it is so de...,False,"[350g/12oz penne pasta, 2 x 80g/3oz packs Parm...",[Cook the pasta in a pan of boiling salted wat...,"[{'ingredient': 'pasta', 'line': '350g/12oz pe...",http://ichef.bbci.co.uk/food/ic/food_16x9_608/...,30,Mary Berry Cooks,p01s4q10,6,1499227763,15 minute pasta,30,http://bbc.co.uk/food/recipes/15_minute_pasta_...
2,Mari Williams,mari_williams,0,Simple 3D iced biscuits inspired by Bake Off. ...,False,"[200g/7oz unsalted soft butter, 200g/7oz caste...","[To make the basic dough, line a baking tray w...","[{'ingredient': 'butter', 'line': '200g/7oz un...",http://ichef.bbci.co.uk/food/ic/food_16x9_608/...,30,The Great British Bake Off,b013pqnm,0,1499227766,3D biscuits,30,http://bbc.co.uk/food/recipes/3d_biscuits_29555
3,Justine Pattison,justine_pattison,0,This easy turkey crown recipe is served with s...,False,[2.5kg/5lb 8oz turkey crown (fully thawed if f...,"[Preheat the oven to 220C/200C Fan/Gas 7., For...","[{'ingredient': 'turkey', 'line': '2.5kg/5lb 8...",http://ichef.bbci.co.uk/food/ic/food_16x9_608/...,30,None,None,6,1499227765,2-hour Christmas dinner,30,http://bbc.co.uk/food/recipes/2_hour_christmas...
7,Delia Smith,delia_smith,30,Delia shows you how to bake a perfect sponge c...,False,"[175g/6oz self-raising flour, 1 rounded tsp ba...","[Preheat the oven to 170C/325F/Gas 3., Take a ...","[{'ingredient': 'self-raising flour', 'line': ...",http://ichef.bbci.co.uk/food/ic/food_16x9_608/...,30,None,None,8,1499227775,A classic sponge cake (with passion fruit fill...,60,http://bbc.co.uk/food/recipes/aclassicspongeca...
18,Rick Stein,rick_stein,0,"Tavë kosi is a national dish in Albania, but I...",False,"[70g/2½oz butter, 1 tbsp olive oil, 1.2kg/2lb ...","[Preheat the oven to 180C/160C Fan/Gas 4., Hea...","[{'ingredient': 'butter', 'line': '70g/2½oz bu...",http://ichef.bbci.co.uk/food/ic/food_16x9_608/...,30,Rick Stein: From Venice to Istanbul,b0667qf6,8,1499227798,Albanian baked lamb with rice (Tavë kosi),30,http://bbc.co.uk/food/recipes/albanian_baked_l...


In [81]:
withphotos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2225 entries, 1 to 10589
Data columns (total 17 columns):
chef                        2225 non-null object
chef_id                     2225 non-null object
cooking_time_minutes        2225 non-null int64
description                 2225 non-null object
error                       2225 non-null bool
ingredients                 2225 non-null object
instructions                2225 non-null object
instructions_detailed       2225 non-null object
photo_url                   2225 non-null object
preparation_time_minutes    2225 non-null int64
program                     1699 non-null object
program_id                  1699 non-null object
serves                      2225 non-null int64
time_scraped                2225 non-null int64
title                       2225 non-null object
total_time_minutes          2225 non-null int64
url                         2225 non-null object
dtypes: bool(1), int64(5), object(11)
memory usage: 297.7+ KB


In [82]:
url_col = withphotos['url']
url_col

1        http://bbc.co.uk/food/recipes/15_minute_pasta_...
2          http://bbc.co.uk/food/recipes/3d_biscuits_29555
3        http://bbc.co.uk/food/recipes/2_hour_christmas...
7        http://bbc.co.uk/food/recipes/aclassicspongeca...
18       http://bbc.co.uk/food/recipes/albanian_baked_l...
                               ...                        
10574    http://bbc.co.uk/food/recipes/yorkshirepudding...
10576    http://bbc.co.uk/food/recipes/yorkshire_tapas_...
10582         http://bbc.co.uk/food/recipes/yule_log_15656
10585    http://bbc.co.uk/food/recipes/zaatar_cod_with_...
10589    http://bbc.co.uk/food/recipes/zesty_tofu_chees...
Name: url, Length: 2225, dtype: object

In [83]:
type(url_col)

pandas.core.series.Series

In [84]:
type(url_col.iloc[0])

str

In [85]:
#^^^debug
url_col.iloc[2097]

'http://bbc.co.uk/food/recipes/toms_turkey_roll_with_44115'

In [86]:
#use regex to clean url:
#example url  'http://bbc.co.uk/food/recipes/3d_biscuits_29555'
    # test regex: \W\w+_(\d+) looks into last segment (incl '/' and finds 29555)

def get_id_from_url(col):
    '''use regex to clean urls to get unique recipe ID for text recipes
    imglist is a list of image title strings
    Returns a dataframe where one column is integer IDs
    the other is the corresponding recipe title''' #may not need
    ID_list = []
    recipe_list = [] 
    for url in col:
        ids = re.findall(r"\W\w+_(\d+)", str(url))
        #print(ids)
        ID_list.append(ids)
        recipes = re.findall(r"\W(\w+)_\d+", str(url))
        recipe_list.append(recipes)
        
    imgID_df = pd.DataFrame(ID_list, columns=['id']) #, title_list,, 'title'])
    imgID_df['short_title'] = recipe_list

    return imgID_df

In [87]:
id_title_df = get_id_from_url(url_col)
id_title_df

,id,short_title
0,33407,[15_minute_pasta]
1,29555,[3d_biscuits]
2,79341,[2_hour_christmas_dinner]
3,9406,[aclassicspongecakewi]
4,92485,[albanian_baked_lamb_with]
...,...,...
2220,86010,[yorkshirepuddings]
2221,93245,[yorkshire_tapas_puddings]
2222,15656,[yule_log]
2223,98478,[zaatar_cod_with_relish]


------
Dec 6 debugging corrupted data after merge:

In [88]:
id_title_df.dtypes

id             object
short_title    object
dtype: object

In [89]:
#ID = 494 and 1495

id_title_df.loc[id_title_df['id']=='494','short_title'] #Pandas indexing!blog?

Series([], Name: short_title, dtype: object)

In [95]:
#^^^debug
id_title_df.loc[494] #by index, not by ID

id                           79152
short_title    [chocolate_roulade]
Name: 494, dtype: object

In [90]:
#reset index to unique ID to locate corrupted row by ID - need to change dtype before assignment??
id_title_df_newID = id_title_df.set_index(['id']) 
id_title_df_newID

,short_title
id,
33407,[15_minute_pasta]
29555,[3d_biscuits]
79341,[2_hour_christmas_dinner]
9406,[aclassicspongecakewi]
92485,[albanian_baked_lamb_with]
...,...
86010,[yorkshirepuddings]
93245,[yorkshire_tapas_puddings]
15656,[yule_log]


In [96]:
#id_title_df_newID.loc[494]

TypeError: cannot do label indexing on <class 'pandas.core.indexes.base.Index'> with these indexers [494] of <class 'int'>

In [97]:
id_title_df_newID.dtypes

short_title    object
dtype: object

In [98]:
id_title_df_newID.index.astype(int)

Int64Index([33407, 29555, 79341,  9406, 92485, 75464, 12416,  3274, 21317,
            86765,
            ...
            91487, 74830, 20002, 69240, 93848, 86010, 93245, 15656, 98478,
            84103],
           dtype='int64', name='id', length=2225)

In [99]:
id_title_df_newID.dtypes

short_title    object
dtype: object

In [109]:

id_title_df.iloc[494] #still by index, not by ID

id                           79152
short_title    [chocolate_roulade]
Name: 494, dtype: object

In [110]:
id_title_df.loc[494] 
#^^^debug

id                           79152
short_title    [chocolate_roulade]
Name: 494, dtype: object

##### Add IDs to doc_topic and labelled topic df

In [84]:
#from section 2.3.4.1: doc_topic = pd.DataFrame(withphotos_tf_nmf_10.round(5),index,columns)
#index = withphotos.index
#columns = ['topic ' + str(num) for num in range(10)]
doc_topic.head()

,topic 0,topic 1,topic 2,topic 3,topic 4,topic 5,topic 6,topic 7,topic 8,topic 9
1,0.00000,0.00000,0.00000,0.00000,0.00000,0.00372,0.00000,0.00000,0.24902,0.12035
2,0.01707,0.03746,0.00612,0.06794,0.00000,0.00000,0.04312,0.00000,0.00000,0.00000
3,0.13109,0.01019,0.00000,0.02241,0.00662,0.04167,0.00000,0.02045,0.02462,0.02640
7,0.00000,0.06281,0.00652,0.01441,0.00045,0.04389,0.09654,0.00000,0.00000,0.00000
18,0.11935,0.00865,0.00292,0.00000,0.00000,0.00399,0.00000,0.07147,0.00792,0.05999


In [85]:
index_ids = id_title_df['id']

doc_id_topic = pd.DataFrame(withphotos_tf_nmf_10.round(5), index=index_ids, columns=columns)
doc_id_topic.head()

,topic 0,topic 1,topic 2,topic 3,topic 4,topic 5,topic 6,topic 7,topic 8,topic 9
id,,,,,,,,,,
33407,0.00000,0.00000,0.00000,0.00000,0.00000,0.00372,0.00000,0.00000,0.24902,0.12035
29555,0.01707,0.03746,0.00612,0.06794,0.00000,0.00000,0.04312,0.00000,0.00000,0.00000
79341,0.13109,0.01019,0.00000,0.02241,0.00662,0.04167,0.00000,0.02045,0.02462,0.02640
9406,0.00000,0.06281,0.00652,0.01441,0.00045,0.04389,0.09654,0.00000,0.00000,0.00000
92485,0.11935,0.00865,0.00292,0.00000,0.00000,0.00399,0.00000,0.07147,0.00792,0.05999


In [86]:
def assign_topic_new(df):
    '''For each row in df, finds the topic with highest weight, 
    assign that topic label to the row.
    Returns a dataframe''' #df easiest to access, will group recipes and images by labels later
    label_list = []
    for i in range(len(df)): #easier way to do this?
        topic_label = np.argmax(np.array(df.transpose().iloc[:,i]), axis = 0)
        label_list.append(topic_label)
    return pd.DataFrame(label_list, index = index_ids, columns = ['topics'])#columns needs to be a list
    #better if make df outside of function?
    #np.argmax(np.array(test), axis = 1)

In [87]:
id_topic_labelled = assign_topic_new(doc_id_topic)
id_topic_labelled

,topics
id,
33407,8
29555,3
79341,0
9406,6
92485,0
...,...
86010,1
93245,1
15656,6


In [88]:
type(id_topic_labelled) 

pandas.core.frame.DataFrame

In [89]:
#Dec 5 - add additional ID column for joining 
id_topic_labelled['ID']=id_topic_labelled.index
id_topic_labelled

,topics,ID
id,,
33407,8,33407
29555,3,29555
79341,0,79341
9406,6,9406
92485,0,92485
...,...,...
86010,1,86010
93245,1,93245
15656,6,15656


##### Add recipe IDs to features.csv

In [90]:
#update Feature_Extractor-Copy1 code: Feature_Extractor_IDs
bbc_ft_id = pd.read_csv('/Users/xinrucheng/Documents/Metis_bootcamp/week_9/metis_passion_project/data/processed/bbc_ft_id.csv')
#Dec 4 10pm updated csv with ID as index

#csv with image features, IDs are assumed to be in the same order as images before feature extraction** 
#(if incorrect, need to extract features again)


In [91]:
bbc_ft_id.head()

,ID,Unnamed: 0,0,1,2,3,4,5,6,7,...,4086,4087,4088,4089,4090,4091,4092,4093,4094,4095
0,33407,0,0.000000,3.509541,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,1.018812,0.000000,2.679488,0.000000
1,79341,1,0.017876,2.738089,0.000000,0.000000,2.499179,0.000000,0.000000,0.000000,...,0.525963,0.000000,0.0,0.0,0.0,3.004055,0.000000,0.000000,4.274214,0.000000
2,29555,2,0.000000,0.000000,0.000000,1.795153,0.000000,1.155841,0.043871,2.865308,...,0.000000,0.000000,0.0,0.0,0.0,0.232908,0.000000,1.179471,0.549198,0.000000
3,19163,3,0.000000,0.468124,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.055124,0.0,0.0,0.0,2.004137,0.000000,0.000000,4.093263,0.000000
4,49934,4,0.000000,0.000000,2.590657,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,3.397941,1.178361,0.598724,2.223511


##### ***Join id_topic (text) and id_features (img) on IDs**

In [92]:
#now both have ID as index, why can't they join?*
#alternative is to add id column for both?

In [93]:
id_topic_labelled.index

Index(['33407', '29555', '79341', '9406', '92485', '75464', '12416', '03274',
       '21317', '86765',
       ...
       '91487', '74830', '20002', '69240', '93848', '86010', '93245', '15656',
       '98478', '84103'],
      dtype='object', name='id', length=2225)

In [94]:
bbc_ft_id.index #not displaying index as above, index or object type not right?

RangeIndex(start=0, stop=2225, step=1)

In [95]:
#bbc_ft_id._set_index('ID') #set index on more time, order slightly different - do they need to be swapped when joining on ID?
#bbc_ft_id.index #should delete ID column by default? still there

In [96]:
bbc_ft_id.head()

,ID,Unnamed: 0,0,1,2,3,4,5,6,7,...,4086,4087,4088,4089,4090,4091,4092,4093,4094,4095
0,33407,0,0.000000,3.509541,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,1.018812,0.000000,2.679488,0.000000
1,79341,1,0.017876,2.738089,0.000000,0.000000,2.499179,0.000000,0.000000,0.000000,...,0.525963,0.000000,0.0,0.0,0.0,3.004055,0.000000,0.000000,4.274214,0.000000
2,29555,2,0.000000,0.000000,0.000000,1.795153,0.000000,1.155841,0.043871,2.865308,...,0.000000,0.000000,0.0,0.0,0.0,0.232908,0.000000,1.179471,0.549198,0.000000
3,19163,3,0.000000,0.468124,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.055124,0.0,0.0,0.0,2.004137,0.000000,0.000000,4.093263,0.000000
4,49934,4,0.000000,0.000000,2.590657,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,3.397941,1.178361,0.598724,2.223511


In [97]:
bbc_ft_id.rename(columns={'Unnamed: 0':'idx'}, inplace=True)#rename column
bbc_ft_id

,ID,idx,0,1,2,3,4,5,6,7,...,4086,4087,4088,4089,4090,4091,4092,4093,4094,4095
0,33407,0,0.000000,3.509541,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,1.018812,0.000000,2.679488,0.000000
1,79341,1,0.017876,2.738089,0.000000,0.000000,2.499179,0.000000,0.000000,0.000000,...,0.525963,0.000000,0.0,0.0,0.0,3.004055,0.000000,0.000000,4.274214,0.000000
2,29555,2,0.000000,0.000000,0.000000,1.795153,0.000000,1.155841,0.043871,2.865308,...,0.000000,0.000000,0.0,0.0,0.0,0.232908,0.000000,1.179471,0.549198,0.000000
3,19163,3,0.000000,0.468124,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.055124,0.0,0.0,0.0,2.004137,0.000000,0.000000,4.093263,0.000000
4,49934,4,0.000000,0.000000,2.590657,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,3.397941,1.178361,0.598724,2.223511
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2220,93848,2220,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.831429,0.658973,0.0,0.0,0.0,2.089992,0.000000,0.000000,5.701145,0.000000
2221,86010,2221,0.000000,0.000000,0.080336,0.000000,0.000000,0.000000,2.034029,0.000000,...,0.000000,2.085165,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
2222,15656,2222,0.000000,2.596903,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.511248,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.010338,0.000000,0.000000
2223,98478,2223,1.356347,1.915114,0.181121,1.021998,0.682499,0.000000,0.000000,1.045778,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000


In [98]:
#bbc_ft_id.drop('idx') #can't drop column after renaming

In [99]:
bbc_ft_id.set_index(['ID'], drop=False)

,ID,idx,0,1,2,3,4,5,6,7,...,4086,4087,4088,4089,4090,4091,4092,4093,4094,4095
ID,,,,,,,,,,,,,,,,,,,,,
33407,33407,0,0.000000,3.509541,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,1.018812,0.000000,2.679488,0.000000
79341,79341,1,0.017876,2.738089,0.000000,0.000000,2.499179,0.000000,0.000000,0.000000,...,0.525963,0.000000,0.0,0.0,0.0,3.004055,0.000000,0.000000,4.274214,0.000000
29555,29555,2,0.000000,0.000000,0.000000,1.795153,0.000000,1.155841,0.043871,2.865308,...,0.000000,0.000000,0.0,0.0,0.0,0.232908,0.000000,1.179471,0.549198,0.000000
19163,19163,3,0.000000,0.468124,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.055124,0.0,0.0,0.0,2.004137,0.000000,0.000000,4.093263,0.000000
49934,49934,4,0.000000,0.000000,2.590657,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,3.397941,1.178361,0.598724,2.223511
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93848,93848,2220,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.831429,0.658973,0.0,0.0,0.0,2.089992,0.000000,0.000000,5.701145,0.000000
86010,86010,2221,0.000000,0.000000,0.080336,0.000000,0.000000,0.000000,2.034029,0.000000,...,0.000000,2.085165,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
15656,15656,2222,0.000000,2.596903,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.511248,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.010338,0.000000,0.000000


In [100]:
#pd.merge(id_topic_labelled, bbc_ft_id, on='ID', how='right')
#ValueError: You are trying to merge on object and int64 columns. If you wish to proceed you should use pd.concat

In [101]:
#bbc_ft_id.rename(columns={'ID':'id'}, inplace=True)#rename column
#bbc_ft_id

In [102]:
id_topic_labelled.dtypes 

topics     int64
ID        object
dtype: object

In [103]:
bbc_ft_id.dtypes

ID        int64
idx       int64
0       float64
1       float64
2       float64
         ...   
4091    float64
4092    float64
4093    float64
4094    float64
4095    float64
Length: 4098, dtype: object

In [104]:
#id_topic_int = id_topic_labelled.astype(int)

In [105]:
#id_topic_int.join(bbc_ft_id, on='ID', how='left', lsuffix='_left', rsuffix='_right') #Join columns with other DataFrame either on index or on a key column.
#join index-on-index by default, ordering? sort? by default=True
#why still NaN? should now share index


In [106]:
#pd.merge(id_topic_labelled.astype(float),bbc_ft_id.astype(float),left_index=True, right_index=True,how='outer') #how='inner' by default
#MergeError: No common columns to perform merge on. Merge options: left_on=None, right_on=None, left_index=False, right_index=False
#astype didn't help? stuck
#when set index = True, how=inner, no overlap (get empty df)

###### Problematic line

In [107]:
#this line with merging caused corrupted rows! **get 2270 when should have 2225
#*debug by trying to join 2 smaller dfs (5rows each) to see what's happening
combined = pd.merge(id_topic_labelled.astype(int), bbc_ft_id, on='ID', how='inner') #expect inner join to give fewer rows?
combined

#start by separating as type and merge*

,topics,ID,idx,0,1,2,3,4,5,6,...,4086,4087,4088,4089,4090,4091,4092,4093,4094,4095
0,8,33407,0,0.000000,3.509541,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.00000,0.0,0.000000,1.018812,0.000000,2.679488,0.000000
1,3,29555,2,0.000000,0.000000,0.000000,1.795153,0.000000,1.155841,0.043871,...,0.000000,0.000000,0.0,0.00000,0.0,0.232908,0.000000,1.179471,0.549198,0.000000
2,0,79341,1,0.017876,2.738089,0.000000,0.000000,2.499179,0.000000,0.000000,...,0.525963,0.000000,0.0,0.00000,0.0,3.004055,0.000000,0.000000,4.274214,0.000000
3,6,9406,5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.492087,...,0.239919,3.340010,0.0,2.14826,0.0,0.000000,2.192654,0.000000,0.489918,0.000000
4,0,92485,6,0.000000,1.017035,0.000000,0.000000,0.510078,0.000000,0.000000,...,0.000000,0.000000,0.0,0.00000,0.0,2.544566,0.000000,0.000000,4.973426,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2265,1,86010,2221,0.000000,0.000000,0.080336,0.000000,0.000000,0.000000,2.034029,...,0.000000,2.085165,0.0,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
2266,1,93245,2219,0.000000,4.411434,0.000000,0.000000,1.053507,0.000000,0.605391,...,2.162922,0.000000,0.0,0.00000,0.0,2.092675,0.000000,0.000000,1.310158,0.000000
2267,6,15656,2222,0.000000,2.596903,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.511248,0.000000,0.0,0.00000,0.0,0.000000,0.000000,0.010338,0.000000,0.000000
2268,5,98478,2223,1.356347,1.915114,0.181121,1.021998,0.682499,0.000000,0.000000,...,0.000000,0.000000,0.0,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000


In [108]:
id_topic_labelled['ID'] = id_topic_labelled['ID'].astype(int)
id_topic_labelled

,topics,ID
id,,
33407,8,33407
29555,3,29555
79341,0,79341
9406,6,9406
92485,0,92485
...,...,...
86010,1,86010
93245,1,93245
15656,6,15656


In [109]:
debug_merged = pd.merge(id_topic_labelled, bbc_ft_id, on='ID', how='inner') #expect inner join to give fewer rows?
debug_merged

,topics,ID,idx,0,1,2,3,4,5,6,...,4086,4087,4088,4089,4090,4091,4092,4093,4094,4095
0,8,33407,0,0.000000,3.509541,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.00000,0.0,0.000000,1.018812,0.000000,2.679488,0.000000
1,3,29555,2,0.000000,0.000000,0.000000,1.795153,0.000000,1.155841,0.043871,...,0.000000,0.000000,0.0,0.00000,0.0,0.232908,0.000000,1.179471,0.549198,0.000000
2,0,79341,1,0.017876,2.738089,0.000000,0.000000,2.499179,0.000000,0.000000,...,0.525963,0.000000,0.0,0.00000,0.0,3.004055,0.000000,0.000000,4.274214,0.000000
3,6,9406,5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.492087,...,0.239919,3.340010,0.0,2.14826,0.0,0.000000,2.192654,0.000000,0.489918,0.000000
4,0,92485,6,0.000000,1.017035,0.000000,0.000000,0.510078,0.000000,0.000000,...,0.000000,0.000000,0.0,0.00000,0.0,2.544566,0.000000,0.000000,4.973426,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2265,1,86010,2221,0.000000,0.000000,0.080336,0.000000,0.000000,0.000000,2.034029,...,0.000000,2.085165,0.0,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
2266,1,93245,2219,0.000000,4.411434,0.000000,0.000000,1.053507,0.000000,0.605391,...,2.162922,0.000000,0.0,0.00000,0.0,2.092675,0.000000,0.000000,1.310158,0.000000
2267,6,15656,2222,0.000000,2.596903,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.511248,0.000000,0.0,0.00000,0.0,0.000000,0.000000,0.010338,0.000000,0.000000
2268,5,98478,2223,1.356347,1.915114,0.181121,1.021998,0.682499,0.000000,0.000000,...,0.000000,0.000000,0.0,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000


In [116]:
dbg=pd.merge(id_topic_labelled, bbc_ft_id, on='ID', how='inner',left_index=False, right_index=False) 
dbg.dtypes

topics      int64
ID          int64
idx         int64
0         float64
1         float64
           ...   
4091      float64
4092      float64
4093      float64
4094      float64
4095      float64
Length: 4099, dtype: object

In [117]:
dbg.shape

(2270, 4099)

In [ ]:
#If `on` is None and not merging on indexes then this defaults to the intersection of the columns in both DataFrames.
#only intersecting column should be ID?

^debug

-------

In [108]:
#convert topics and id back to int?
#combined['topic_int'] = combined.topics.astype(int)
#combined['id'] = combined.id.astype(int)
#combined.head()

In [109]:
combined.drop(['idx'],axis=1) #why is idx 6? thought that recipe had no image*


,topics,ID,0,1,2,3,4,5,6,7,...,4086,4087,4088,4089,4090,4091,4092,4093,4094,4095
0,0.0,33407.0,0.000000,3.509541,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.00000,0.0,0.000000,1.018812,0.000000,2.679488,0.000000
1,3.0,29555.0,0.000000,0.000000,0.000000,1.795153,0.000000,1.155841,0.043871,2.865308,...,0.000000,0.000000,0.0,0.00000,0.0,0.232908,0.000000,1.179471,0.549198,0.000000
2,7.0,79341.0,0.017876,2.738089,0.000000,0.000000,2.499179,0.000000,0.000000,0.000000,...,0.525963,0.000000,0.0,0.00000,0.0,3.004055,0.000000,0.000000,4.274214,0.000000
3,6.0,9406.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.492087,0.000000,...,0.239919,3.340010,0.0,2.14826,0.0,0.000000,2.192654,0.000000,0.489918,0.000000
4,7.0,92485.0,0.000000,1.017035,0.000000,0.000000,0.510078,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.00000,0.0,2.544566,0.000000,0.000000,4.973426,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2265,7.0,86010.0,0.000000,0.000000,0.080336,0.000000,0.000000,0.000000,2.034029,0.000000,...,0.000000,2.085165,0.0,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
2266,1.0,93245.0,0.000000,4.411434,0.000000,0.000000,1.053507,0.000000,0.605391,0.000000,...,2.162922,0.000000,0.0,0.00000,0.0,2.092675,0.000000,0.000000,1.310158,0.000000
2267,6.0,15656.0,0.000000,2.596903,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.511248,0.000000,0.0,0.00000,0.0,0.000000,0.000000,0.010338,0.000000,0.000000
2268,5.0,98478.0,1.356347,1.915114,0.181121,1.021998,0.682499,0.000000,0.000000,1.045778,...,0.000000,0.000000,0.0,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000


In [ ]:
#keep as float, usable?*
#how to index by ID? reset index of combined again?

In [ ]:
#pd.merge(df_a, df_b, on='subject_id', how='left') #if no match to a in b, output null in that cell
#pd.merge(bbc_ft_id, id_topic_labelled, on='id', how='left')

In [ ]:
#pd.concat([id_topic_labelled, bbc_ft_id], axis=1)


In [ ]:
#id_topic_labelled.join(bbc_ft_id, how='inner')#join using indexes
#default how = left, join using topic frame's index (default)


#### Check matched IDs make sense for a few random recipes

In [110]:
combined.iloc[-1]

topics        1.000000
ID        84103.000000
idx        2224.000000
0             0.399728
1             0.824599
              ...     
4091          0.000000
4092          5.132298
4093          0.000000
4094          3.680424
4095          4.284847
Name: 2269, Length: 4099, dtype: float64

From 2.3.5.1, 10 topics


```
* Topic 0 roast - potatoes, lamb, beef, pork
* Topic 1 dessert - chocolate, meringue
* Topic 2 dessert - pastry, pie, tart
* Topic 3 bread 
* Topic 4 chicken
* Topic 5 salad
* Topic 6 cake
* Topic 7 Asian - curry, rice, stir-fry
* Topic 8 pasta
* Topic 9 pan fry
```

Recipe ID 84103 is zesty_tofu_cheesecake_84103_16x9.jpg

    - should be dessert or cake (1,2,6)
    
 Assigned to topic 1, ok?

In [111]:
combined.iloc[4]

topics        7.000000
ID        92485.000000
idx           6.000000
0             0.000000
1             1.017035
              ...     
4091          2.544566
4092          0.000000
4093          0.000000
4094          4.973426
4095          0.000000
Name: 4, Length: 4099, dtype: float64

Albanian baked lamb classified as topic 8 which is pasta, not great, but could be due to not having enough topics
(pasta could be baked?)

Compare match results from 2 dataframes with just ID and title:

img_ids  and id_title_df

In [112]:
#automate check?
img_ids.head()

,ID,title
0,33407,15_minute_pasta_33407_16x9.jpg
1,79341,2_hour_christmas_dinner_79341_16x9.jpg
2,29555,3d_biscuits_29555_16x9.jpg
3,19163,_chicken_chasseur_with_19163_16x9.jpg
4,49934,_schichttorte_49934_16x9.jpg


In [113]:
id_title_df.head() #from 2.3.7.2, ids and titles extracted from text url

,id,short_title
0,33407,[15_minute_pasta]
1,29555,[3d_biscuits]
2,79341,[2_hour_christmas_dinner]
3,9406,[aclassicspongecakewi]
4,92485,[albanian_baked_lamb_with]


IDs 33407 and 29555 seem to match, good

Also searched in image directory and 92485 is indeed albanian_baked_lamb, consistent with id_title_df from url

Can automate this check for larger dataset (later)

In [114]:
combined.iloc[2267]

topics        6.000000
ID        15656.000000
idx        2222.000000
0             0.000000
1             2.596903
              ...     
4091          0.000000
4092          0.000000
4093          0.010338
4094          0.000000
4095          0.000000
Name: 2267, Length: 4099, dtype: float64

In [115]:
#are indices idx consistent? idx from image df, can use to check  (if write separate ftn)
img_ids.iloc[2222]  #same ID, good!

ID                         15656
title    yule_log_15656_16x9.jpg
Name: 2222, dtype: object

Yule log is a cake, classified as topic 6, good!

#### Get images from selected topic category

In [119]:
def get_recipes_by_topic_ID(num, data):
    '''Takes in the topic number (0 to 9),
    returns a list of IDs of all recipe titles assigned to that topic
    (those that have the label as the topic with maximum weight from NMF topic modelling)'''
    docs_same_label = []
    #result = [f(x) for x in df['col']] better and faster than iter ftns
    #result = [f(x, y) for x, y in zip(df['col1'], df['col2'])]
    for topicn, idn in zip(data['topics'], data['ID']): #change id to ID 
        if topicn == num:
            docs_same_label.append(idn)
    return docs_same_label

In [168]:
cakes_list = get_recipes_by_topic(6.0, combined)
#a list of all recipes IDs that are cakes
len(cakes) #still 150 here, where did 150->153 happen?

150

In [121]:
cakes_list[0]

9406.0

In [123]:
combined_ididx = combined.set_index(['ID'])
combined_ididx #2270 rows! expecting 2225

,topics,idx,0,1,2,3,4,5,6,7,...,4086,4087,4088,4089,4090,4091,4092,4093,4094,4095
ID,,,,,,,,,,,,,,,,,,,,,
33407.0,0.0,0.0,0.000000,3.509541,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.00000,0.0,0.000000,1.018812,0.000000,2.679488,0.000000
29555.0,3.0,2.0,0.000000,0.000000,0.000000,1.795153,0.000000,1.155841,0.043871,2.865308,...,0.000000,0.000000,0.0,0.00000,0.0,0.232908,0.000000,1.179471,0.549198,0.000000
79341.0,7.0,1.0,0.017876,2.738089,0.000000,0.000000,2.499179,0.000000,0.000000,0.000000,...,0.525963,0.000000,0.0,0.00000,0.0,3.004055,0.000000,0.000000,4.274214,0.000000
9406.0,6.0,5.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.492087,0.000000,...,0.239919,3.340010,0.0,2.14826,0.0,0.000000,2.192654,0.000000,0.489918,0.000000
92485.0,7.0,6.0,0.000000,1.017035,0.000000,0.000000,0.510078,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.00000,0.0,2.544566,0.000000,0.000000,4.973426,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86010.0,7.0,2221.0,0.000000,0.000000,0.080336,0.000000,0.000000,0.000000,2.034029,0.000000,...,0.000000,2.085165,0.0,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
93245.0,1.0,2219.0,0.000000,4.411434,0.000000,0.000000,1.053507,0.000000,0.605391,0.000000,...,2.162922,0.000000,0.0,0.00000,0.0,2.092675,0.000000,0.000000,1.310158,0.000000
15656.0,6.0,2222.0,0.000000,2.596903,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.511248,0.000000,0.0,0.00000,0.0,0.000000,0.000000,0.010338,0.000000,0.000000


In [125]:
testrow0 = combined_ididx.loc[33407] #use loc for index here (by value) instead of iloc
testrow0

topics    0.000000
idx       0.000000
0         0.000000
1         3.509541
2         0.000000
            ...   
4091      0.000000
4092      1.018812
4093      0.000000
4094      2.679488
4095      0.000000
Name: 33407.0, Length: 4098, dtype: float64

In [126]:
type(testrow0)  #pasta classified as roast (topic 0), might be a case where clustering would help?*

pandas.core.series.Series

In [127]:
testrow3 = combined_ididx.loc[9406] #use loc for index here instead of iloc
testrow3

topics    6.000000
idx       5.000000
0         0.000000
1         0.000000
2         0.000000
            ...   
4091      0.000000
4092      2.192654
4093      0.000000
4094      0.489918
4095      0.000000
Name: 9406.0, Length: 4098, dtype: float64

In [128]:
#cake classified as cake still (topic 6), good

In [ ]:
#need to put all the ftns together in main pipeline

In [129]:
#need to write another ftn to get entire row with features in df, not just ID list?

#find all the cakes
def features_by_topic(idlist, combineddf):
    '''Takes in an id list from get_recipes_by_topic,
    finds the features of images in that list, save in a dataframe'''
    ftgroup = []
    for recipeid in idlist:
        ftrow = np.array(combineddf.loc[recipeid])
        ftgroup.append(ftrow)
    return pd.DataFrame(ftgroup)

cake_features_df = features_by_topic(cakes_list, combined_ididx) 
cake_features_df

,0,1,2,3,4,5,6,7,8,9,...,4088,4089,4090,4091,4092,4093,4094,4095,4096,4097
0,6,5,0,0,0.000000,0.0,0.000000,0.000000,2.492087,0.000000,...,0.239919,3.340010,0.0,2.148260,0.000000,0.000000,2.192654,0.000000,0.489918,0.0
1,6,9,0,0.274039,0.000000,0.0,0.000000,0.000000,1.724523,0.000000,...,0.000000,0.000000,0.0,0.938444,0.000000,0.979124,1.231072,0.034272,0.000000,0.0
2,6,10,0,0,0.000000,0.0,0.000000,0.000000,0.488151,0.192400,...,0.000000,1.431639,0.0,0.573350,0.000000,0.000000,0.000000,0.000000,1.317004,0.0
3,6,17,0,0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,3.772132,0.0,1.373650,0.000000,0.489011,0.000000,0.000000,0.179028,0.0
4,6,19,0,1.63672,0.000000,0.0,0.101847,0.000000,0.396604,0.000000,...,0.000000,0.000000,0.0,0.000000,0.992393,0.000000,0.000000,0.000000,2.321203,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,6,2169,0,0,0.000000,0.0,0.000000,0.000000,0.000000,3.245846,...,0.000000,2.356380,0.0,0.266896,0.000000,0.000000,0.000000,0.000000,1.831635,0.0
149,6,2172,0,0.82622,0.000000,0.0,0.000000,0.000000,0.438433,0.000000,...,0.202663,0.903240,0.0,4.131788,0.000000,0.000000,1.666926,0.000000,2.423386,0.0
150,6,2194,0,0,0.370191,0.0,0.514775,1.910061,4.864564,0.000000,...,2.072880,1.667925,0.0,0.000000,0.000000,0.000000,5.029452,0.041010,0.000000,0.0
151,6,2191,0,0.0116296,0.446989,0.0,0.294821,2.410504,0.849241,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,1.202494,0.000000,2.293362,0.000000,0.0


#### Find cosine similarity within topic

In [ ]:
#def cos_sim_by_topic(imgft, topicft):
    '''Takes in features for one image (row in df), 
    and a df of all images in the same topic group
    return the cosine similarity (one vs all) as an array'''
#can use prev ftn?    

In [130]:
#from cosine_similarity notebook:
def cos_sim_vs_all(imageft, datasetft):
    '''Find the pairwise cosine similarity between features of the chosen image and all the images in the dataset'''
    sim_list = []
    for i in range(len(datasetft)):
        cos_sim = cosine_similarity(imageft, datasetft.iloc[i].values.reshape(1,-1)) #output of cosine_sim is already an array here
        sim_list.append(cos_sim)
        #print(np.stack(sim_list, axis=0).shape)
    #return sim_list
    return np.stack(sim_list, axis=0) #use stack to convert a list of arrays to one array
#convert ft df to matrix?*

#get error when using np.array to convert output to array: https://stackoverflow.com/questions/4674473/valueerror-setting-an-array-element-with-a-sequence

In [131]:
#compare first cake with the rest of the cakes

cos_sim_vs_all(cake_features_df.iloc[0].values.reshape(1,-1), cake_features_df.head())  #use iloc for df indices!

array([[[1.        ]],

       [[0.5753688 ]],

       [[0.50661623]],

       [[0.46837556]],

       [[0.41748737]]])

In [142]:
sim_cake0 = cos_sim_vs_all(cake_features_df.iloc[0].values.reshape(1,-1), cake_features_df.head(40)) 
#get old error on larger df? fixed for small dataset

#problem with cake_features_df, before cosine similarity
#*debug--by going through different number of rows, see where it crashes, good up to row 40 in cake_features, row 41 corrupted?

In [145]:
cake_features_df.iloc[39]

0             6
1           465
2             0
3       2.72996
4             0
         ...   
4093          0
4094    3.06167
4095          0
4096    1.37959
4097          0
Name: 39, Length: 4098, dtype: object

In [146]:
cake_features_df.iloc[0]

0              6
1              5
2              0
3              0
4              0
          ...   
4093           0
4094     2.19265
4095           0
4096    0.489918
4097           0
Name: 0, Length: 4098, dtype: object

In [147]:
cake_features_df.iloc[150]

0               6
1            2194
2               0
3               0
4        0.370191
          ...    
4093            0
4094      5.02945
4095    0.0410097
4096            0
4097            0
Name: 150, Length: 4098, dtype: object

In [143]:
cake_features_df.iloc[40] #index mismatch after this? could be hard to fix

0       [6.0, 494.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.8...
1       [6.0, 1495.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.14146...
2       [5.0, 494.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.8...
3       [5.0, 1495.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.14146...
4                                                     NaN
                              ...                        
4093                                                  NaN
4094                                                  NaN
4095                                                  NaN
4096                                                  NaN
4097                                                  NaN
Name: 40, Length: 4098, dtype: object

In [144]:
cake_features_df.iloc[41] #just these 2 rows? transposed? recipe ID 494 and 1495, check in feature matrix?

0       [6.0, 494.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.8...
1       [6.0, 1495.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.14146...
2       [5.0, 494.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.8...
3       [5.0, 1495.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.14146...
4                                                     NaN
                              ...                        
4093                                                  NaN
4094                                                  NaN
4095                                                  NaN
4096                                                  NaN
4097                                                  NaN
Name: 41, Length: 4098, dtype: object

In [150]:
cake_features_df.iloc[42]

0               6
1             495
2               0
3               0
4        0.436465
          ...    
4093     0.697132
4094     0.101824
4095            0
4096    0.0972521
4097     0.126291
Name: 42, Length: 4098, dtype: object

In [148]:
cake_features_df.iloc[149]

0             6
1          2172
2             0
3       0.82622
4             0
         ...   
4093          0
4094    1.66693
4095          0
4096    2.42339
4097          0
Name: 149, Length: 4098, dtype: object

In [149]:
cake_features_df.iloc[140]

0             6
1          2086
2             0
3             0
4             0
         ...   
4093          0
4094    2.26688
4095          0
4096          0
4097          0
Name: 140, Length: 4098, dtype: object

In [156]:
#check large df of features for all images, how many NaNs?
cake_features_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 153 entries, 0 to 152
Columns: 4098 entries, 0 to 4097
dtypes: float64(4094), object(4)
memory usage: 4.8+ MB


In [158]:
cake_features_df.describe()

,4,5,6,7,8,9,10,11,12,13,...,4088,4089,4090,4091,4092,4093,4094,4095,4096,4097
count,147.000000,147.000000,147.000000,147.000000,147.000000,147.000000,147.000000,147.000000,147.000000,147.000000,...,147.000000,147.000000,147.000000,147.000000,147.000000,147.000000,147.000000,147.000000,147.000000,147.000000
mean,0.138306,0.196955,0.306323,0.278115,1.271333,0.453859,0.310347,0.855819,0.014650,0.126931,...,0.785164,1.054000,0.041524,0.817231,0.199240,0.223808,0.792708,0.214912,0.942807,0.400255
std,0.493999,0.627865,0.690406,0.768469,1.486258,0.937399,0.757086,1.539012,0.148727,0.413419,...,1.192526,1.447391,0.225336,1.244994,0.512637,0.527468,1.273527,0.530906,1.339174,0.787239
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.666706,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.128557,0.394728,0.000000,0.000000,0.000000,0.000000,0.022898,0.000000,0.271816,0.000000
75%,0.000000,0.000000,0.166531,0.000000,2.127231,0.331795,0.065030,1.144676,0.000000,0.000000,...,1.273832,1.687366,0.000000,1.101956,0.000000,0.000000,1.186786,0.000000,1.343614,0.310251
max,3.222694,5.178796,4.264771,4.579515,6.595136,5.218029,5.473833,7.591406,1.763126,2.885091,...,6.635208,6.637444,1.455257,5.639992,2.634250,2.496529,6.256621,3.081371,6.430125,4.297850


In [155]:
#only 147 rows with values, 3 rows corrupted? 41 42 and which one?
len(cake_features_df) #also after rerunning kernel, len 153 instead of 150, so now 6 bad rows?

153

In [166]:
for row in cake_features_df:
    if cake_features_df.iloc[row].isnull().any()==True:
        print(row) #maybe instead of returning old idx, want ID? if not a single int or float, corrupted (like in idx 40 above)

40
41
96
97
102
103


IndexError: single positional indexer is out-of-bounds

In [ ]:
#3 pairs of consecutive rows are corrupted - how? how to fix? might have more in other topics
#problem seems to be with large df after combining and resetting index?

In [169]:
combined_ididx.info()

<class 'pandas.core.frame.DataFrame'>
Float64Index: 2270 entries, 33407.0 to 84103.0
Columns: 4098 entries, topics to 4095
dtypes: float64(4098)
memory usage: 71.1 MB


In [172]:
bad_list = []
for i in combined_ididx.iloc['idx']:
    if combined_ididx.iloc[i].isnull().any()==True:
        bad_list.append(i)
bad_list

TypeError: Cannot index by location index with a non-integer key

In [ ]:
#bug before cos sim, Join_on_IDs_debug.ipynb

In [160]:
np.array(cake_features_df.iloc[40]).transpose()

array([array([6.00000000e+00, 4.94000000e+02, 0.00000000e+00, ...,
       0.00000000e+00, 3.29863286e+00, 6.16847277e-02]),
       array([   6., 1495.,    0., ...,    0.,    0.,    0.]),
       array([5.00000000e+00, 4.94000000e+02, 0.00000000e+00, ...,
       0.00000000e+00, 3.29863286e+00, 6.16847277e-02]),
       ..., nan, nan, nan], dtype=object)

----

*If after filtering by topic, cos similarity results not good, consider going back to doc-topic matrix and perform clustering instead of argmax

-----

### Topic modelling with LDA -- skip

In [169]:
#https://www.analyticsvidhya.com/blog/2016/08/beginners-guide-to-topic-modeling-in-python/
#also, LDA exercise lecture notebook

In [170]:
import gensim
from gensim import corpora, models, similarities, matutils
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import string

#### Text cleaning and preprocessing

In [171]:
# stop = set(stopwords.words('english'))
# exclude = set(string.punctuation)
# lemma = WordNetLemmatizer()
# def clean(doc):
#     stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
#     punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
#     normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
#     return normalized

# doc_clean = [clean(doc).split() for doc in doc_complete]  

In [172]:
# #create a Gensim dictionary from the texts
# dictionary = corpora.Dictionary(texts)

# #remove extremes (similar to the min/max df step used when creating the tf-idf matrix)
# dictionary.filter_extremes(no_below=1, no_above=0.8)

# #convert the dictionary to a bag of words corpus for reference
# corpus = [dictionary.doc2bow(text) for text in texts]

##### stopwords

In [173]:
#use nltk stopwords
nltkstopwords = stopwords.words('english')
len(nltkstopwords)

179

In [174]:
#custom stop_words
nltkstopwords.extend(['minutes','add','remove'])
len(nltkstopwords)

182

In [175]:
#follow lecture notebook, use countvectorizer, then convert to Gensim
#analyzer=word: Tokenize by word
#ngram_range=(1,2): Keep all 1 and 2-word grams
#stop_words=english: Remove all English stop words
#token_pattern=\\b[a-z][a-z]+\\b: Match all tokens with 2 or more (strictly) alphabet characters


#Create a CountVectorizer for parsing/counting words
count_vectorizer = CountVectorizer(ngram_range=(1, 2),  
                                   stop_words=nltkstopwords, token_pattern="\\b[a-z][a-z]+\\b")

In [176]:
count_vectorizer.fit(cld_withphotos)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 2), preprocessor=None,
                stop_words=['i', 'me', 'my', 'myself', 'we', 'our', 'ours',
                            'ourselves', 'you', "you're", "you've", "you'll",
                            "you'd", 'your', 'yours', 'yourself', 'yourselves',
                            'he', 'him', 'his', 'himself', 'she', "she's",
                            'her', 'hers', 'herself', 'it', "it's", 'its',
                            'itself', ...],
                strip_accents=None, token_pattern='\\b[a-z][a-z]+\\b',
                tokenizer=None, vocabulary=None)

In [177]:
# Create the term-document matrix
# Transpose it so the terms are the rows(for gensim?)
doc_word = count_vectorizer.transform(cld_withphotos).transpose()
doc_word.shape #without transpose, document-term

(101747, 2225)

In [178]:
pd.DataFrame(doc_word.toarray(), count_vectorizer.get_feature_names()).head()

,0,1,2,3,4,5,6,7,8,9,...,2215,2216,2217,2218,2219,2220,2221,2222,2223,2224
abd,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
abd boiled,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
able,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
able cover,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
able hold,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [179]:
#convert SciPy matrix to Gensim friendly corpus:
corpus = matutils.Sparse2Corpus(doc_word)

Map matrix rows to words (tokens)

save a mapping (dict) of row id to word (token) for later use by gensim:

In [180]:
id2word = dict((v, k) for k, v in count_vectorizer.vocabulary_.items())
len(id2word)

101747

#### Creating an LDA model
It requires our corpus of word counts, mapping of row ids to words, and the number of topics (3).

In [181]:
#without additional custom stopwords, LDA topics not as distinct as NMF ones, also have some strange phrases
#went back to edit stopwords, to remove minutes, add, remove

#also try TF-IDF vectorization?

In [182]:
lda = models.LdaModel(corpus=corpus, num_topics=20, id2word=id2word, passes=5) #slow? ~1min

In [183]:
lda.print_topics()
#10 most important words for 20 topics, 5 passes

[(0,
  '0.005*"jars" + 0.003*"brandy rum" + 0.002*"sauce little" + 0.002*"sterilised" + 0.002*"sterilised jars" + 0.001*"zester" + 0.001*"based saucepan" + 0.001*"pan leaves" + 0.001*"base pan" + 0.001*"sterilise"'),
 (1,
  '0.017*"dough" + 0.009*"bowl" + 0.009*"icing" + 0.008*"baking" + 0.007*"sugar" + 0.007*"place" + 0.006*"chocolate" + 0.006*"surface" + 0.006*"roll" + 0.006*"mixture"'),
 (2,
  '0.002*"leave chicken" + 0.001*"vanilla pods" + 0.001*"chop butter" + 0.001*"large dish" + 0.001*"dust cocoa" + 0.001*"processor whiz" + 0.001*"whiz smooth" + 0.001*"carefully hot" + 0.001*"half biscuits" + 0.001*"job taste"'),
 (3,
  '0.011*"pastry" + 0.011*"pudding" + 0.008*"basin" + 0.008*"job" + 0.007*"water" + 0.005*"flour" + 0.005*"foil" + 0.005*"dough" + 0.005*"kid" + 0.005*"kid job"'),
 (4,
  '0.003*"stock absorbed" + 0.002*"ladleful" + 0.002*"ladleful stock" + 0.001*"little freshly" + 0.001*"flour cover" + 0.001*"soft rice" + 0.001*"everything coated" + 0.001*"walnut oil" + 0.001*"ano

20 topics, 5 passes

0. dessert - pastry
1. dessert - cake
2. dessert - cream, mascarpone
3. chicken salad
4. ? pan, heat, sugar
5. beef, yorkshire pudding
6. ? sieving flour
7. dessert - pudding
8. fishcakes
9. fish chowder
10. fish fillets
11. rice and chicken stir fry
12. ? veg and fish
13. ? lentils and fish
14. ? eggs and stock
15. savoury tart? bacon and onions
16. ? pan, heat, oil
17. baked chicken, fish
18. soy, icing? dessert?
19. ? japanese mayo and mulled wine

**LDA is stochastic! will get different topics each time we run it!** -- problematic for assigning documents to topics

In [184]:
lda20_10passes = models.LdaModel(corpus=corpus, num_topics=20, id2word=id2word, passes=10)
#compare 10 passes with 5 passes above, do more passes give better topics?
lda20_10passes.print_topics()

[(0,
  '0.001*"sauce spaghetti" + 0.001*"duck legs" + 0.001*"duck fat" + 0.000*"hot enough" + 0.000*"reserved duck" + 0.000*"duck meat" + 0.000*"beans spinach" + 0.000*"parfait" + 0.000*"check oil" + 0.000*"give batter"'),
 (1,
  '0.017*"oil" + 0.014*"pan" + 0.012*"heat" + 0.008*"fry" + 0.006*"cook" + 0.006*"bowl" + 0.006*"salt" + 0.005*"frying" + 0.005*"frying pan" + 0.005*"drain"'),
 (2,
  '0.013*"place" + 0.012*"oven" + 0.008*"salt" + 0.007*"oil" + 0.007*"bowl" + 0.007*"mix" + 0.007*"pepper" + 0.007*"chicken" + 0.005*"serve" + 0.005*"season"'),
 (3,
  '0.003*"spoon yoghurt" + 0.001*"bruise" + 0.001*"almond milk" + 0.001*"browned put" + 0.001*"make vinaigrette" + 0.001*"pour top" + 0.001*"squeeze fresh" + 0.000*"clam" + 0.000*"help create" + 0.000*"soup mixture"'),
 (4,
  '0.032*"cake" + 0.023*"cm" + 0.011*"cm cake" + 0.010*"icing" + 0.009*"board" + 0.008*"cake board" + 0.008*"top" + 0.006*"cakes" + 0.005*"repeat" + 0.005*"around"'),
 (5,
  '0.015*"lime" + 0.013*"stir fry" + 0.011*"w

20 topics, 10 passes - might be too many? But topic #17 is roast chicken and turkey!

0. hamburger (assemble with kids)
1. ?
2. beef, egg batter
3. dessert - butter, yogurt, fruit and jam
4. indian
5. dessert - cake
6. ?
7. dessert - pastry
8. salad/dessert
9. grill/bbq
10. chips and dip
11. tacos?
12. thai noodles
13. ? corn and stevia
14. baking
15. ? boiled
16. ? pickles
17. roast turkey and chicken!
18. ?
19. ? similar to 18


In [185]:
lda20_15passes = models.LdaModel(corpus=corpus, num_topics=20, id2word=id2word, passes=15) #very slow, stuck?
#do more passes give better topics?
lda20_15passes.print_topics()

[(0,
  '0.038*"cake" + 0.019*"icing" + 0.013*"cm" + 0.009*"sugar" + 0.009*"tin" + 0.009*"top" + 0.007*"chocolate" + 0.007*"beat" + 0.006*"using" + 0.006*"cakes"'),
 (1,
  '0.004*"pancakes" + 0.003*"pancake" + 0.002*"cook golden" + 0.001*"cm non" + 0.001*"pancakes warm" + 0.001*"milk food" + 0.001*"remaining batter" + 0.001*"repeat remaining" + 0.001*"pancake cook" + 0.001*"keep cooked"'),
 (2,
  '0.002*"spoon yoghurt" + 0.001*"smooth alternatively" + 0.001*"onion chopped" + 0.001*"lard large" + 0.001*"sprinkle fresh" + 0.001*"curry sprinkle" + 0.001*"blitz smooth" + 0.001*"dark rye" + 0.001*"stir coated" + 0.001*"lentils saucepan"'),
 (3,
  '0.000*"ml floz" + 0.000*"floz" + 0.000*"version" + 0.000*"torn" + 0.000*"salady" + 0.000*"seedless delectably" + 0.000*"chopped tending" + 0.000*"quarters chunky" + 0.000*"tending towards" + 0.000*"black olives"'),
 (4,
  '0.004*"griddle" + 0.003*"iron" + 0.003*"cakes" + 0.003*"iron griddle" + 0.003*"splash milk" + 0.002*"welsh cakes" + 0.002*"wels

In [186]:
lda_10 = models.LdaModel(corpus=corpus, num_topics=10, id2word=id2word, passes=20) 

In [187]:
lda_10.print_topics()#20 most important words for 10 topics

[(0,
  '0.003*"duck" + 0.002*"dripping" + 0.001*"parts" + 0.001*"beef dripping" + 0.001*"pancakes" + 0.001*"red onions" + 0.001*"parts spring" + 0.001*"onions potatoes" + 0.001*"pour tall" + 0.001*"tomatoes olives"'),
 (1,
  '0.009*"job" + 0.006*"kid job" + 0.006*"kid" + 0.004*"adult" + 0.003*"adult job" + 0.002*"vegetables" + 0.002*"lemon" + 0.002*"want" + 0.002*"like" + 0.002*"crumble"'),
 (2,
  '0.003*"jars" + 0.002*"jam" + 0.001*"place ribs" + 0.001*"boil" + 0.001*"sugar" + 0.001*"boil cook" + 0.001*"evaporated" + 0.001*"almost evaporated" + 0.001*"almost" + 0.001*"hot"'),
 (3,
  '0.004*"croquettes" + 0.001*"croquette" + 0.001*"top frying" + 0.001*"flour milk" + 0.001*"pasta broccoli" + 0.001*"top grated" + 0.001*"tomatoes using" + 0.001*"whiz" + 0.001*"cheese place" + 0.001*"melted season"'),
 (4,
  '0.013*"heat" + 0.012*"pan" + 0.010*"oil" + 0.009*"cook" + 0.007*"salt" + 0.007*"stir" + 0.007*"pepper" + 0.006*"water" + 0.006*"fry" + 0.006*"serve"'),
 (5,
  '0.007*"pasta" + 0.002*"

Not many repeated words across 10 topics which is good. Can probably have more topics? 

Topics: (*get different topics for diff number of passes - when does it converge?)

0. ? cocktail (crepes? special char not english)
1. kid-friendly
2. ice lolly
3. dessert - pastry
4. dessert - pastry
5. dessert - chocolate, meringue
6. teriyaki
7. ? pan
8. dessert - cake
9. ? corn and game meat?

20 topics seem better than 10, use lda above

In [188]:
#* can set a large number (50 or 100) to make sure it converges, then compare 20 with 10 topics!

##### 20 topics, 100 passes

*Gensim can run in parallel? 100 and 50 passes both >5min30s, converged

In [189]:
lda20_100passes = models.LdaModel(corpus=corpus, num_topics=20, id2word=id2word, passes=100) #very slow, stuck?
lda20_100passes.print_topics() #compare with 5,10,15 passes above?

[(0,
  '0.012*"pan" + 0.011*"heat" + 0.011*"pasta" + 0.011*"cook" + 0.007*"sauce" + 0.006*"stir" + 0.005*"large" + 0.005*"gently" + 0.005*"put" + 0.004*"frying"'),
 (1,
  '0.007*"fillets" + 0.004*"beurre" + 0.004*"haddock" + 0.003*"blanc" + 0.003*"beurre blanc" + 0.003*"smoked haddock" + 0.003*"walnuts" + 0.003*"chowder" + 0.002*"haddock fillets" + 0.002*"lamb"'),
 (2,
  '0.014*"mixture" + 0.011*"sugar" + 0.010*"pastry" + 0.008*"oven" + 0.008*"bowl" + 0.007*"tin" + 0.007*"cream" + 0.007*"egg" + 0.007*"whisk" + 0.006*"together"'),
 (3,
  '0.013*"place" + 0.013*"oven" + 0.010*"salt" + 0.009*"pepper" + 0.008*"oil" + 0.008*"bowl" + 0.007*"mix" + 0.007*"season" + 0.006*"well" + 0.006*"preheat"'),
 (4,
  '0.002*"topping ingredients" + 0.002*"remaining fruit" + 0.002*"slice bread" + 0.001*"back cling" + 0.001*"another slice" + 0.001*"evaporated sugar" + 0.001*"peel back" + 0.001*"beans top" + 0.001*"eggs using" + 0.001*"boil cook"'),
 (5,
  '0.000*"matchstick" + 0.000*"matchstick potatoes" + 

##### 10 topics, 100 passes

In [ ]:
lda10_100passes = models.LdaModel(corpus=corpus, num_topics=10, id2word=id2word, passes=100) #very slow, stuck?
lda10_100passes.print_topics() 

In [ ]:
#LDA is problematic because topics change each time the code is run; go back to use NMF topics to assign labels to recipes

#### Map documents to Topic space, using LDA Transformer

In [ ]:
# Transform the docs from the word space to the topic space (like "transform" in sklearn)
#lda_corpus = lda[corpus]
#lda_corpus

In [ ]:
# Store the documents' topic vectors in a list
#lda_docs = [doc for doc in lda_corpus]

In [ ]:
#document vectors in the topic space -- measures of the component of each document along each topic
# Check out the document vectors in the topic space for the first 5 documents 
#(out of 20 topics, expect most will have fewer than 3)  #10 passes
#lda_docs[0:5]

In this case, first document is 19 or 18 (they have similar words), 2nd is most similar to topic 5, 3rd is topic 18, 4th is topic 5, 5th is topic 19

In [192]:
#check actual documents to see if results make sense
cld_withphotos.head()#just instructions

1       cook the pasta in a pan of boiling salted wa...
2       to make the basic dough  line a baking tray ...
3       preheat the oven to c c fan gas      for the...
7       preheat the oven to c f gas      take a very...
18      preheat the oven to c c fan gas      heat g ...
Name: instructions, dtype: object

In [193]:
withphotos.title.head()

1                                       15 minute pasta
2                                          3D biscuits 
3                               2-hour Christmas dinner
7     A classic sponge cake (with passion fruit fill...
18            Albanian baked lamb with rice (Tavë kosi)
Name: title, dtype: object

the 2nd and 4th recipe in the subset with photos are indeed desserts (topic 5) (20 topics, 10 passes?)

In [199]:
withphotos.title

1                                          15 minute pasta
2                                             3D biscuits 
3                                  2-hour Christmas dinner
7        A classic sponge cake (with passion fruit fill...
18               Albanian baked lamb with rice (Tavë kosi)
                               ...                        
10574                                   Yorkshire puddings
10576                           Yorkshire ‘tapas’ puddings
10582                                             Yule log
10585                             Za’atar cod with relish 
10589                                Zesty tofu cheesecake
Name: title, Length: 2225, dtype: object

----

### Clustering on topics - Assign image to topics/cluster?

Dec. 3
*Roberto: after getting document-topic matrix, can either assign document to topic with highest weight, or cluster in topic space (documents are points in that space)! 

In [ ]:
#kmeans clustering:
# from sklearn.externals import joblib

# #uncomment the below to save your model 
# #since I've already run my model I am loading from the pickle

# #joblib.dump(km,  'doc_cluster.pkl')

# km = joblib.load('doc_cluster.pkl')
# clusters = km.labels_.tolist()
#http://brandonrose.org/clustering - also has LDA example

    #for clustering on topics, may need cos sim/euc dist of topics first?

In [ ]:
#try kmeans (or DBSCAN?) on local or AWS - separate from topic results? ask
    #assign each image to cluster, then find cluster of test image, find cosine similarity (0.5-1day?)
    #pickle and move to Flask

also:
https://datascience.stackexchange.com/questions/19991/cluster-documents-based-on-topic-similarity/19999?noredirect=1#comment23532_19999
> Topics are clusters
> There is next to no difference between subspace clustering and topic modeling, except maybe that text is sparse and integer while subspace clusterers usually assume dense and continuous data.
So rather than trying to cluster again, just use your topics.
Yes, documents can belong to multiple topics. That is because text usually is this way, and forcing everything to have a unique label reduces the quality, because it cannot reflect reality anymore.
 If you insist every document should have a unique cluster, just use argmax(topic weights).



> (why topic modelling is not used for classification) Topic modeling produces a number of topics for tokens in your text. Overall, you end up with a number of different topics (even per sentence) and in lots of texts there will be multiple prominent topic with high scores. That doesn't help you determine which class the text belongs to overall.
https://www.quora.com/Why-isnt-topic-modelling-used-more-for-document-classification

topic modelling with LDA example https://www.analyticsvidhya.com/blog/2016/08/beginners-guide-to-topic-modeling-in-python/ *

In [ ]:
#argmax(topic weights) - from topic modelling output? get topic weights for new image, then cos sim
    #would need to assign each of the 2k images to its top topic
#also under topic modelling lecture: LDA+gensim example notebook

#if use topic modelling and LDA, don't actually make use of the 10 topics I already have (repeating work?)
    #didn't use them for prediction in project4 either, want to improve on that

In [ ]:
#(Also, how to incorporate this with user uploaded info (ingredient or cuisine type?)--may not need that if sep into topics/clusters?)